In [7]:
import igraph as ig
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.stats.multitest import multipletests

# Functions and code snippets

In [8]:
def union(list1,list2): # performs a union operation between both lists passed as arguments
    unique_list = []
    for element in list1:
        if element not in unique_list:
            unique_list.append(element)
        else:
            continue
    for element in list2:
        if element not in unique_list:
            unique_list.append(element)
        else:
            continue
    return unique_list

def unique(doubled_list):
    unique_list = []
    for element in doubled_list:
        if element in unique_list:
            continue
        unique_list.append(element)
    return unique_list

def exclude(list1,list2): # retrieve elements in list1 that are not present in list2
    # list1 - list2
    return unique([element for element in list1 if element not in list2])

def intersect(list1,list2): # intersection between two sets of values
    intersection_list = list()
    for element in list1:
        if element in list2 and element not in intersection_list:
            intersection_list.append(element)

    return unique([element for element in list1 if element in list2 and element])

def value_count(my_list):
    list_count = {}
    for index in range(len(my_list)):
        count = 0
        for element in list:
            if element == my_list[index]:
                count += 1
        list_count[my_list[index]] = count
    
    return list_count

#______________________________________________________________________________

def add_interactors(graph,subgraph,int_list,attribute:str,show_statistic=False):
    maingraph_intersect = intersect(graph.vs[attribute],int_list) # intersection between interactors and the general graph
    int_not_in_graph = exclude(int_list,maingraph_intersect)
    print(f"{len(int_not_in_graph)} experimental interactors have no interaction described in the current model")
    subgraph_intersect = intersect(subgraph.vs[attribute],int_list) # intersection between interactors and the specific graph
    interactors_to_add = exclude(maingraph_intersect,subgraph_intersect) # remove already existing interactors
    seed_set = which_attribute(union(maingraph_intersect,subgraph_intersect),graph,"name")
    if show_statistic:
        cent_measure = np.array(subgraph.betweenness())[seed_set]
        fig, ax = plt.subplots(1,1,figsize=(7,5))

        ax.boxplot(cent_measure)
        plt.show()
    new_int = {}
    i = 0
    for interactor in interactors_to_add:
        index = graph.vs.find(name=interactor).index # get the interactor index
        name = graph.vs[index][attribute]
  
        neighbors = graph.neighbors(index,mode="ALL") # retrieve the interactor's neighbors in the general graph
        neig_in_subgraph = intersect(subgraph.vs[attribute],graph.vs[neighbors][attribute]) # check which neighbors are already in the subgraph

        if not neig_in_subgraph:
            print(f"{graph.vs[index][attribute]} has no neighbor in the selected subgraph")
            continue
        else:
            subgraph.add_vertices(name)
            edge_list = []
            for neighbor in neig_in_subgraph:
                edge_list.append((name,neighbor))
            subgraph.add_edges(edge_list)
            i += 1
        new_int[name] = neig_in_subgraph
    print(f"{i} nodes were added")
    return new_int

#______________________________________________________________________________
# function to obtain the index of the biggest component of the current graph
def which_max(graph):
    biggest_component = max([len(x) for x in graph.components()])
    for element in graph.components():
        if len(element) == biggest_component:
            index = list(graph.components()).index(element)
            return index
        
def which_index(node_list,graph,attribute_name:str): # retrieve the attribute of the nodes in a list based on a specific index in the graph
    attribute_list = []
    for node in node_list:
        attribute_list.append(graph.vs[node][attribute_name])
    return attribute_list

def which_attribute(attribute_list,graph,attribute): # retrieve the indexes of the nodes in a list based on a specific attribute in the graph
    node_index = [v.index for v in graph.vs if v[attribute] in attribute_list]
    return node_index

def get_seeds(seed_list,graph,percentile=95):
    centralities = pd.Series(graph.degree())[seed_list]
    threshold = np.percentile(centralities,percentile)
    filtered_seeds = centralities[centralities < threshold].index.to_list()
    return filtered_seeds


# Import APID database and create a network graph from it

In [9]:
# Read APID database file
apid = pd.read_csv("9606_noISI_Q2.txt",sep="\t")
epac1 = "O95398" # EPAC1 uniprot ID
cftr = "P13569" # CFTR uniprot ID
ezr = "P15311"
nherf1 = "O14745" #"O14745" 
capza2 = "P47755"
inf2 = "Q27J81"

apid

,InteractionID,UniprotID_A,UniprotName_A,GeneName_A,UniprotID_B,UniprotName_B,GeneName_B,ExpEvidences,Methods,Publications,3DStructures,CurationEvents
0,1205000,Q14160,SCRIB_HUMAN,SCRIB,B7Z2Y1,B7Z2Y1_HUMAN,NaN,1,1,1,0,3
1,1205001,Q14160,SCRIB_HUMAN,SCRIB,Q14155,ARHG7_HUMAN,ARHGEF7,11,8,8,0,20
2,1205002,Q14160,SCRIB_HUMAN,SCRIB,Q7Z628,ARHG8_HUMAN,NET1,2,2,2,0,2
3,1205003,P22460,KCNA5_HUMAN,KCNA5,Q14160,SCRIB_HUMAN,SCRIB,1,1,1,0,2
4,1205004,Q96DN2,VWCE_HUMAN,VWCE,Q14160,SCRIB_HUMAN,SCRIB,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
265211,2429577,P50570,DYN2_HUMAN,DNM2,Q6VMQ6,MCAF1_HUMAN,ATF7IP,1,1,1,0,1
265212,2429588,Q2TAL8,QRIC1_HUMAN,QRICH1,Q6VMQ6,MCAF1_HUMAN,ATF7IP,1,1,1,0,1
265213,2429629,P10114,RAP2A_HUMAN,RAP2A,P04049,RAF1_HUMAN,RAF1,1,1,1,0,1
265214,2429638,Q7L591,DOK3_HUMAN,DOK3,P07949,RET_HUMAN,RET,1,1,1,0,1


In [10]:
all_uniprots = pd.concat([apid["UniprotID_A"],apid["UniprotID_B"]],ignore_index=True)
all_names = pd.concat([apid["GeneName_A"],apid["GeneName_B"]],ignore_index=True)

uniprot_to_name = pd.concat([all_uniprots,all_names],axis=1)

id_to_name_map = dict()

def isnone(object):
    if type(object) == type(np.NaN):
        return True
    else:
        return False

for uniprotid,name in zip(uniprot_to_name.iloc[:,0],uniprot_to_name.iloc[:,1]):
    if not isnone(name):
        id_to_name_map[uniprotid] = name
    else:
        id_to_name_map[uniprotid] = "No Name"

map = pd.DataFrame({"UniprotID":id_to_name_map.keys(),"Gene Name":id_to_name_map.values()})
"""map = map.groupby("Gene Name").agg(list).reset_index() # group duplicate gene names
map["UniprotID"] = map["UniprotID"].apply(lambda x: x[0] if isinstance(x,list) else x)""" # select the first uniprot ID"""
#map.to_csv("UniprotID_Genename_mapping.csv")


# Note: some uniprot IDS represent the same gene name, but the annotation is deffect in the gene name field so it's better to leave it as it is
# And check the results through the gene name


'map = map.groupby("Gene Name").agg(list).reset_index() # group duplicate gene names\nmap["UniprotID"] = map["UniprotID"].apply(lambda x: x[0] if isinstance(x,list) else x)'

### Warning: multiple UniprotIDs in APID make reference to the same gene, meaning that there is a glitch in both the topology of the resulting graph and the real degree of a gene node (since some interactions are associated to the Swiss-Prot ID and others with another ID). This may have influence in PPR diffusion since it is proportional to the degree.

In [11]:
all_nodes = pd.concat([apid["UniprotID_A"],apid["UniprotID_B"]]).unique() # df with all nodes 

mapping = {node : idx for idx, node in enumerate(all_nodes)} # create a map for every uniprot ID and each index in the df for graph creation

In [12]:
#apid_uniprot = apid.iloc[:,[1,4]].replace(mapping) # replace the uniprot id for the numerical index

In [13]:
apid_uniprot = pd.read_csv("Apid_graph_table.csv")

In [14]:
big_graph = ig.Graph(edges=list(zip(apid_uniprot["UniprotID_A"], apid_uniprot["UniprotID_B"]))) # create a graph from the previous df

big_graph.vs["name"] = list(all_nodes) # insert the uniprot IDs as attributes of each node within the graph

print(big_graph.summary())

index = which_max(big_graph)

IGRAPH UN-- 18173 265216 -- 
+ attr: name (v)


In [15]:
big_graph_simp = big_graph.copy()

big_graph_simp.simplify() # simplify the previous graph to remove self loops and multiple edges

print(big_graph_simp.summary())

IGRAPH UN-- 18173 262728 -- 
+ attr: name (v)


## HPA and GTEx transcriptomic data to filter tissue-specific genes

In [16]:
tissue_proteome = pd.read_csv("rna_tissue_consensus.tsv",sep='\t')
lung_genes = tissue_proteome.loc[(tissue_proteome["Tissue"] == "lung") & (tissue_proteome["nTPM"] != 0)]["Gene name"].to_list()

In [17]:
uni2gene = pd.read_csv("UniprotID_Genename_mapping.csv").iloc[:,1:]

In [18]:
import mygene

# Initialize the query object
mg = mygene.MyGeneInfo()

# Query mygene.info for UniProt ID mappings
#gene_info = mg.querymany(cf, scopes=['ensembl.gene','symbol'], fields='uniprot', species='human')

# Extract GeneID to UniProt mapping
results = mg.querymany(
    lung_genes,
    scopes=["ensembl.gene", "symbol"],
    fields="uniprot.Swiss-Prot,symbol,name",
    species="human",
    returnall=True
)
lung_df = pd.DataFrame(results["out"]) # get the output of the conversion
lung_df = lung_df[["query", "symbol", "name", "uniprot"]] # select useful columns
lung_df["SwissProt"] = lung_df["uniprot"].apply(lambda x: x.get("Swiss-Prot") if isinstance(x, dict) else None) # get the swiss-prot ID for each protein
lung_df["SwissProt"] = lung_df["SwissProt"].apply(lambda x: x[0] if type(x) == list else x) # if it is doubled, select the first entry
# Frontiers in Pharmacology 

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:querying 1001-2000...
INFO:biothings.client:done.
INFO:biothings.client:querying 2001-3000...
INFO:biothings.client:done.
INFO:biothings.client:querying 3001-4000...
INFO:biothings.client:done.
INFO:biothings.client:querying 4001-5000...
INFO:biothings.client:done.
INFO:biothings.client:querying 5001-6000...
INFO:biothings.client:done.
INFO:biothings.client:querying 6001-7000...
INFO:biothings.client:done.
INFO:biothings.client:querying 7001-8000...
INFO:biothings.client:done.
INFO:biothings.client:querying 8001-9000...
INFO:biothings.client:done.
INFO:biothings.client:querying 9001-10000...
INFO:biothings.client:done.
INFO:biothings.client:querying 10001-11000...
INFO:biothings.client:done.
INFO:biothings.client:querying 11001-12000...
INFO:biothings.client:done.
INFO:biothings.client:querying 12001-13000...
INFO:biothings.client:done.
INFO:biothings.client:querying 13001-14000...
INFO:biothings

In [19]:
print(f"There are {sum(lung_df.apply(lambda x: True if type(x) == list else False))} duplicate entry")

There are 0 duplicate entry


In [20]:
missing_genes_all = results["missing"] # check which genes had no equivalent uniprot ID

missing_genes = uni2gene.loc[uni2gene["Gene Name"].isin(missing_genes_all)]["UniprotID"].to_list() # force the conversion through the APID's database mapping

print(f"{len(missing_genes_all) - len(missing_genes)} genes were not retrieved")

28 genes were not retrieved


In [21]:
lung_genes_list = unique(lung_df.dropna(axis=0,subset="SwissProt")["SwissProt"].to_list() + missing_genes) # total lung genes uniprot ids
#print(len(lung_genes_list))
graph_genes = big_graph_simp.vs["name"] # retrieve all apid uniprot ids

genes_to_keep = intersect(graph_genes,lung_genes_list) # genes to keep

genes_to_remove = [gene for gene in graph_genes if gene not in genes_to_keep] # subtraction between graph_genes & genes_to_keep 
        
if genes_to_remove:
    print(f"{len(genes_to_remove)} genes are not expressed in lung tissue")

big_graph_simp.delete_vertices([big_graph_simp.vs.find(name=uniprotid) for uniprotid in genes_to_remove])

2993 genes are not expressed in lung tissue


In [22]:
graph_main = big_graph_simp.induced_subgraph(big_graph_simp.components()[index]) # choose the component of the simplified graph that has the greatest amount of proteins in it

print(graph_main.summary())
#graph_main.write_graphml("graph_main.graphml") # save a separate file for graph_main


IGRAPH UN-- 15105 229186 -- 
+ attr: name (v)


# DISGENET network for specific diseases

In [23]:
disgenet = pd.read_csv("all_gene_disease_associations.tsv",sep='\t')
cf = disgenet.loc[(disgenet["diseaseName"] == "Cystic Fibrosis")]
cf = unique(cf["geneSymbol"].to_list())
disgenet#.loc[(disgenet["geneSymbol"]=="EZR") & (disgenet["diseaseName"] == "Cystic Fibrosis")]

# check transcriptomic data to filter genes
# getex
# human protein atlas

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.700,0.538,C0001418,Adenocarcinoma,group,C04,Neoplastic Process,0.01,1.0,2008.0,2008.0,1,0,LHGDN
1,1,A1BG,0.700,0.538,C0002736,Amyotrophic Lateral Sclerosis,disease,C18;C10,Disease or Syndrome,0.01,1.0,2008.0,2008.0,1,0,BEFREE
2,1,A1BG,0.700,0.538,C0003578,Apnea,phenotype,C23;C08,Sign or Symptom,0.01,1.0,2017.0,2017.0,1,0,BEFREE
3,1,A1BG,0.700,0.538,C0003864,Arthritis,disease,C05,Disease or Syndrome,0.01,1.0,2019.0,2019.0,1,0,BEFREE
4,1,A1BG,0.700,0.538,C0008373,Cholesteatoma,disease,C17,Disease or Syndrome,0.01,1.0,2020.0,2020.0,1,0,BEFREE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134937,115804232,CEROX1,NaN,NaN,C0005890,Body Height,phenotype,NaN,Organism Attribute,0.10,1.0,2019.0,2019.0,1,0,GWASCAT
1134938,115891964,MIR223HG,0.861,0.077,C0023418,leukemia,disease,C04,Neoplastic Process,0.01,1.0,2016.0,2016.0,1,0,BEFREE
1134939,115891964,MIR223HG,0.861,0.077,C0023467,"Leukemia, Myelocytic, Acute",disease,C04,Neoplastic Process,0.01,1.0,2016.0,2016.0,1,0,BEFREE
1134940,115891964,MIR223HG,0.861,0.077,C0598766,Leukemogenesis,disease,C23;C04,Neoplastic Process,0.01,1.0,2016.0,2016.0,1,0,BEFREE


## Find the equivalent UniprotID for the ENTREZ gene ID

In [24]:
cf_uniprot = uni2gene.loc[map["Gene Name"].isin(cf)]["UniprotID"] # convert CF gene symbols to uniprot IDs

print(f"{len(cf_uniprot)} genes were converted")

"SLC9A3R1" in map["Gene Name"]

799 genes were converted


False

In [25]:
import mygene

# Initialize the query object
mg = mygene.MyGeneInfo()

# Query mygene.info for UniProt ID mappings
gene_info = mg.querymany(cf, scopes=[ "symbol"], fields="uniprot.Swiss-Prot,symbol,name", species='human', returnall=True)
results = pd.DataFrame(gene_info["out"])
results["Swiss-Prot"] = results["uniprot"].apply(lambda x: x.get("Swiss-Prot") if isinstance(x,dict) else x)
results["Swiss-Prot"] = results["Swiss-Prot"].apply(lambda x: x[0] if isinstance(x,list) else x)
results = results.dropna(axis=0,subset="Swiss-Prot")
print(f"{len(results)} genes were converted")

INFO:biothings.client:querying 1-852...
INFO:biothings.client:done.
INFO:biothings.client:Finished.


787 genes were converted


In [26]:
mg_cf_uniprot = results["Swiss-Prot"].to_list()

total_conversion = union(mg_cf_uniprot,cf_uniprot)

print(f"{len(total_conversion)} genes")

843 genes


In [27]:
total_conversion = [gene for gene in total_conversion if gene != None]
total_conversion = [gene[0] if type(gene) == list else gene for gene in total_conversion] # UNIPROT IDs for the CF-associated gene list

print(f"There are {len(total_conversion)} equivalent UniproIDs")

There are 843 equivalent UniproIDs


In [28]:
vertex_list = graph_main.vs["name"]
print(f"There are {len(intersect(vertex_list,total_conversion))} genes presents in the main graph")
def which_list(test_list,main_list): # function to retrieve the indexes of the entries contained within a main list
    index_list = [main_list.index(element) for element in test_list if element in main_list]    
    return index_list

cf_vertex = which_list(total_conversion,vertex_list) # get the index of the proteins that appear in the main graph and are CF-related

There are 687 genes presents in the main graph


# Create a subgraph with CF-relevant proteins only

In [29]:
cf_graph = graph_main.induced_subgraph(cf_vertex)
print(cf_graph.summary())

IGRAPH UN-- 687 1811 -- 
+ attr: name (v)


In [30]:
#cf_graph.write_graphml("graph.graphml")

In [31]:
cf_index = which_max(cf_graph) # retrieve the index of the largest subcomponent of the new graph
biggest_cf_component = cf_graph.components()[cf_index] # largest CF-associated connected component 
print(f"The biggest CF-associated graph component has {len(biggest_cf_component)} connected proteins")

The biggest CF-associated graph component has 526 connected proteins


# Connect the CF-related network by solving the minimum Steiner tree problem

In [32]:
def joincomp(idlist, g):
    """
    Translates the R joincomp function to Python using igraph.

    Args:
        idlist: A list of vertex IDs (integers) to connect.
        g: An igraph Graph object.

    Returns:
        A dictionary containing:
            'subg': The induced subgraph with added Steiner nodes.
            'uni': A list of unique Steiner node IDs added.
            'all': A list of lists, where each inner list contains possible Steiner node IDs for a connection.
            'addtable': A pandas DataFrame with node counts and neighbor counts.
            'mst': A pandas DataFrame representing the MST edges.
    """
    # Get the shortest path between all nodes of interest
    subdist = g.shortest_paths(idlist, idlist) # calculates the shortest path between all node of interest
    closure = ig.Graph.Weighted_Adjacency(subdist, mode="undirected") # creates a new graph from the previous variable: the nodes are the IDs present and there are edges between all of the nodes (their respective weight is the shortest path between the pair of nodes)
    # Note: the indexes are reseted in "closure"
    clos_mst = closure.spanning_tree(weights=closure.es["weight"])  # find a minimum spanning tree from the nodes in "closure"
    clos_mst_table = pd.DataFrame(clos_mst.get_edgelist(), columns=["from", "to"]) # extract the edges to a dataframe
    clos_mst_table["weight"] = clos_mst.es["weight"] # adds the weight to the df

    # variable initialization
    vnames = g.vs["name"]  # Gets the UNIPROT id (the name attribute) of the nodes
    id1 = []
    id2 = []
    all2add = [] # grouped Steiner nodes for each MST edge
    allvec = [] # continuous list of Steiner nodes
    listind = 0

    # Find the possible Steiner nodes that compose the shortest paths (and that can be added to connect the initial graph)
        # Get the names of each node in the mst graph
    for i in range(len(clos_mst_table)):
        from_name = clos_mst_table["from"][i] # the mst graph index of the ith element
        to_name = clos_mst_table["to"][i] # the mst index of the ith target
        id1_val = vnames.index(vnames[idlist[from_name]]) # retrieve the original idlist indexes
        id2_val = vnames.index(vnames[idlist[to_name]])

        id1.append(id1_val)
        id2.append(id2_val)

        # Retrieve the shortest paths between each pair of nodes in the mst graph
        if clos_mst_table["weight"][i] > 1: # for nodes that are not directly connected...
            sp_res = g.get_all_shortest_paths(v=id1_val, to=id2_val) # ...finds all shortests paths between these nodes                       
            temp2add = []
            for sp in sp_res: # for each shortest path...
                spj = [int(node) for node in sp]
                temp2add.append(spj[1:-1]) # add separately each connecting path to a list (excluding the first and last nodes)
                allvec.extend(spj[1:-1])    # add all connecting nodes together to a continuous list
            all2add.append(temp2add) # add the shortest-path-nodes to the list
            listind += 1

    steiner_count = pd.Series(allvec).value_counts() # counts all Steiner nodes instances

    # counts how many times each Steiner node appear
    addtable = pd.DataFrame(index=range(len(steiner_count)),data={ # df with unique Steiner nodes
        'node':steiner_count.index.to_list(),
        'count':steiner_count.to_list()
    }) 

    # counts how many neighbors each Steiner node has in the subgraph of interest
    addtable["neighbor_count"] = addtable["node"].apply(lambda node: len(set(g.neighbors(node)).intersection(idlist))) 

    uni2add = []
    for i in range(len(all2add)): # for each shortest path collection with length > 1...
        if len(all2add[i]) > 1: # ...if there is more than 1 shortest path connecting the node pair...
            alt_mat = pd.DataFrame(index=range(len(all2add[i])), columns=["count_sum", "neighbor_sum"]) # df in which rows are the shortest paths 
            for j in range(len(all2add[i])): # for each shortest path in all2add[i]...
                alt_mat["count_sum"][j] = addtable[addtable["node"].isin(all2add[i][j])]["count"].sum() # scores the shortest path with the Steiner nodes within the jth shortest path for the ith edge on clos_mst_table in terms of overall count
                alt_mat["neighbor_sum"][j] = addtable[addtable["node"].isin(all2add[i][j])]["neighbor_count"].sum() # scores the shortest path based on Steiner node's neighbor count in the subgraph of interest

            alt_choice = alt_mat.sort_values(by=["count_sum", "neighbor_sum"], ascending=False).index.tolist()
            uni2add.append(all2add[i][alt_choice[0]]) # for each MST edge retrieves the highest scoring shortest path
        else:
            uni2add.append(all2add[i][0]) # if there is only one shortest path, retrieve this one

    uni2add = unique([item for sublist in uni2add for item in sublist]) 
    subnodes = unique(idlist + uni2add)
    subg = g.induced_subgraph(subnodes)
    clos_mst_table["id1"] = id1
    clos_mst_table["id2"] = id2

    return {"subg": subg, "uni": uni2add, "all": all2add, "addtable": addtable, "mst": clos_mst_table}

In [33]:
cf_connected = joincomp(cf_vertex,graph_main) # performs the Steriner algorith to connect the CF network

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\1428738538.py:18: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead
  subdist = g.shortest_paths(idlist, idlist) # calculates the shortest path between all node of interest
C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\1428738538.py:71: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the ca

In [34]:
cf_connected_graph = cf_connected['subg']

cf_connected_graph.summary()


'IGRAPH UN-- 813 3539 -- \n+ attr: name (v)'

In [35]:
for node in cf_connected_graph.vs["name"]:
    print(node)


P35222
Q8TBB1
P04439
P01889
P07900
P16333
P51681
Q9UHD4
P12004
P05787
Q12888
P04637
Q99836
P07355
P02787
P04004
P08581
P10415
P02545
Q13547
Q92793
Q6NUS6
Q96Q45
Q9P0N5
P42336
P0CG48
Q03518
P51610
P42345
P31749
P01009
P08246
P31946
P02741
P01308
O95433
P05019
P46108
Q07666
P98170
O15151
P19320
P07585
P05156
P01019
P50052
P25101
P24530
P00533
P24394
P25025
Q16539
Q06609
P45983
Q16665
P02649
P55072
O60603
O00206
P00519
P61769
P30153
P22303
P21802
P15336
Q05655
Q13557
Q9Y2R2
P10145
P15692
P38398
Q8IVH4
O00187
Q12778
P38646
P21980
P17936
P29466
P60484
O14920
P25106
P35348
P14618
P04626
O14745
P13569
O00585
Q9Y4X3
P13501
P69905
Q92743
P03372
P04150
P11473
Q15185
P31948
Q03933
O95817
P04792
P16615
P30988
P51788
Q99933
Q92993
Q96Q40
Q9UBN7
Q9UL15
P04040
P10809
P15311
O14880
Q15118
Q9UKV5
P14555
P25490
P49407
P15884
P08151
Q15399
P02724
O60602
Q9NR96
P17676
P35638
P17861
P17252
Q03519
P35869
P61586
P06731
P62913
P38936
Q9ULZ3
Q09472
Q05086
Q9UHD9
Q13283
Q9Y6K9
P62993
P32780
P98082
P78536
O75791

## Centrality measures of the connected CF-related network

In [36]:
degree = cf_connected_graph.degree() # get the degree of each node
betweeness = cf_connected_graph.betweenness() # get the betweenness of each node
closeness = cf_connected_graph.closeness() # get the closeness of each node

cf_centralities_measure = pd.DataFrame(index=range(len(cf_connected_graph.components()[0])),data={ # summary dataframe
    "UniprotID":cf_connected_graph.vs["name"],
    "Degree":degree,
    "Betweeness":betweeness,
    "Closeness":closeness
})

cftr_betweeness = cf_centralities_measure.loc[cf_centralities_measure["UniprotID"] == cftr]["Betweeness"].values[0]
cftr_closeness = cf_centralities_measure.loc[cf_centralities_measure["UniprotID"] == cftr]["Closeness"].values[0]
cftr_degree = cf_centralities_measure.loc[cf_centralities_measure["UniprotID"] == cftr]["Degree"].values[0]

# retrieve CFTR neighbors' centrality scores

cf_neighbors = cf_connected_graph.neighbors(cf_connected_graph.vs.find(name=cftr).index, mode="all")

cf_neighbors = [cf_connected_graph.vs[neighbor]["name"] for neighbor in cf_neighbors] # get the uniprot ID of the neighbors

cf_nei_betweeness = cf_centralities_measure.loc[cf_centralities_measure["UniprotID"].isin(cf_neighbors)]["Betweeness"].values
cf_nei_closeness = cf_centralities_measure.loc[cf_centralities_measure["UniprotID"].isin(cf_neighbors)]["Closeness"].values
cf_nei_degree = cf_centralities_measure.loc[cf_centralities_measure["UniprotID"].isin(cf_neighbors)]["Degree"].values

print(f"Null value inputation to 1")
zero_bet = cf_centralities_measure.loc[cf_centralities_measure["Betweeness"] == 0].index
cf_centralities_measure.loc[zero_bet,"Betweeness"] = 1 # input non-null values to become a manageable number for further processing

cf_centralities_measure.sort_values(axis=0,by="Betweeness",ascending=False)#.loc[cf_centralities_measure["UniprotID"] == cftr]


Null value inputation to 1


,UniprotID,Degree,Betweeness,Closeness
138,P62993,82,20896.170357,0.416410
48,P00533,85,20200.656869,0.419638
144,P42858,77,19710.402042,0.417481
11,P04637,96,16274.340691,0.423358
18,P02545,45,13434.863914,0.394750
...,...,...,...,...
584,P18089,1,1.000000,0.245912
587,Q92583,2,1.000000,0.241164
588,P15813,1,1.000000,0.256637
62,P22303,1,1.000000,0.268519


In [37]:
fig = go.Figure(data=go.Scatter(x=degree,
                                y=closeness,
                                mode='markers',
                                hovertext=cf_connected_graph.vs["name"],
                                name="All nodes"))

fig.update_layout(title=dict(text='Closeness vs. Degree - CF Graph',font=dict(family="Arial",size=25)),
    xaxis_title=dict(text='Degree',font=dict(size=20)),
    yaxis_title=dict(text='Closeness',font=dict(size=20)),
    legend=dict(font=dict(size=18)),
    width=800,  # Width in pixels
    height=600,  # Height in pixels
)

# Highlight CFTR
fig.add_trace(go.Scatter(
    x=[cftr_degree],
    y=[cftr_closeness],
    mode='markers',
    name='CFTR',
    marker=dict(color='red', size=10, symbol='star'),
    hovertext=cf_centralities_measure.loc[cf_centralities_measure["UniprotID"] == cftr]["UniprotID"].values[0]
))

fig.add_trace(go.Scatter(
    x=cf_nei_degree,
    y=cf_nei_closeness,
    mode='markers',
    name='CFTR Neighbors',
    marker=dict(color='green', size=7, symbol='square'),
    hovertext=cf_neighbors
))

fig.show()

In [38]:
fig = go.Figure(data=go.Scatter(x=degree,
                                y=betweeness,
                                mode='markers',
                                hovertext=cf_connected_graph.vs["name"],
                                name="All nodes"))

fig.update_layout(title=dict(text='Betweeness vs. Degree - CF Graph',font=dict(family="Arial",size=25)),
    xaxis_title=dict(text='Degree',font=dict(size=20)),
    yaxis_title=dict(text='Betweenness',font=dict(size=20)),
    legend=dict(font=dict(size=18)),
    width=800,  # Width in pixels
    height=600,  # Height in pixels
)

fig.add_trace(go.Scatter(
    x=[cftr_degree],
    y=[cftr_betweeness],
    mode='markers',
    name='CFTR',
    marker=dict(color='red', size=10, symbol='star'),
    hovertext=cf_centralities_measure.loc[cf_centralities_measure["UniprotID"] == cftr]["UniprotID"].values[0]
))

fig.add_trace(go.Scatter(
    x=cf_nei_degree,
    y=cf_nei_betweeness,
    mode='markers',
    name='CFTR Neighbors',
    marker=dict(color='green', size=7, symbol='square'),
    hovertext=cf_neighbors
))

fig.show()


In [39]:
fig = go.Figure(data=go.Scatter(x=betweeness,
                                y=closeness,
                                mode='markers',
                                hovertext=cf_connected_graph.vs["name"],
                                name="All nodes"))

fig.update_layout(title=dict(text='Closeness vs. Betweenness - CF Graph',font=dict(family="Arial",size=25)),
    xaxis_title=dict(text='Betweenness',font=dict(size=20)),
    yaxis_title=dict(text='Closeness',font=dict(size=20)),
    legend=dict(font=dict(size=18)),
    width=800,  # Width in pixels
    height=600,  # Height in pixels
)

fig.add_trace(go.Scatter(
    x=[cftr_betweeness],
    y=[cftr_closeness],
    mode='markers',
    name='CFTR',
    marker=dict(color='red', size=10, symbol='star'),
    hovertext=cf_centralities_measure.loc[cf_centralities_measure["UniprotID"] == cftr]["UniprotID"].values[0]
))

fig.add_trace(go.Scatter(
    x=cf_nei_betweeness,
    y=cf_nei_closeness,
    mode='markers',
    name='CFTR Neighbors',
    marker=dict(color='green', size=7, symbol='square'),
    hovertext=cf_neighbors
))

fig.add_trace(go.Scatter(
    x=cf_nei_betweeness,
    y=cf_nei_closeness,
    mode='markers',
    name='CFTR Neighbors',
    marker=dict(color='green', size=7, symbol='square'),
    hovertext=cf_neighbors
))

fig.show()

In [40]:
# Betweeness distribution
f, ax = plt.subplots(1,2,figsize=(14,5))

ax[0].hist(cf_centralities_measure["Betweeness"], bins=30)
ax[0].set_title("Betweeness Distribution")
ax[0].set_xlabel("Betweeness")
ax[0].set_ylabel("Frequency")

ax[1].hist(cf_nei_betweeness, bins=30)
ax[1].set_title("CFTR Neighbors Betweeness Distribution")
ax[1].set_xlabel("Betweeness")
ax[1].set_ylabel("Frequency")

plt.show()

# Degree distribution
f, ax = plt.subplots(1,2,figsize=(14,5))

ax[0].hist(cf_centralities_measure["Degree"], bins=30)
ax[0].set_title("Degree Distribution")
ax[0].set_xlabel("Degree")
ax[0].set_ylabel("Frequency")

ax[1].hist(cf_nei_degree, bins=20)
ax[1].set_title("CFTR Neighbors Degree Distribution")
ax[1].set_xlabel("Degree")
ax[1].set_ylabel("Frequency")

plt.show()

# Closeness distribution
f, ax = plt.subplots(1,2,figsize=(14,5))

ax[0].hist(cf_centralities_measure["Closeness"], bins=30)
ax[0].set_title("Closeness Distribution")
ax[0].set_xlabel("Closeness")
ax[0].set_ylabel("Frequency")

ax[1].hist(cf_nei_closeness, bins=10)
ax[1].set_title("CFTR Neighbors Closeness Distribution")
ax[1].set_xlabel("Closeness")
ax[1].set_ylabel("Frequency")

plt.show()

In [41]:
import numpy as np
import matplotlib.pyplot as plt
graph_main_degree = graph_main.degree()
degrees = np.array(cf_centralities_measure["Degree"]) # 
degrees = degrees[degrees > 0]  # remove zero degrees to avoid log issues

# Get unique degrees and their counts
unique_deg, counts = np.unique(degrees, return_counts=True)

# Plot in log–log scale
plt.figure(figsize=(7,5))
plt.scatter(unique_deg, counts, marker='o', s=30)
plt.xscale("log")
#plt.yscale("log")
plt.xlabel("Degree")
plt.ylabel("Frequency")
plt.title("Degree Distribution (log–log)")
#plt.grid(True, which="both", ls="--", alpha=0.5)
plt.show()


## Find specific modules within the connected graph

In [42]:
#cf_cluster = cf_connected_graph.community_multilevel() # inspect clusters 7. 8 and 9
#cf_cluster = cf_connected_graph.community_spinglass()
#cf_cluster = cf_connected_graph.community_leading_eigenvector() 
#cf_cluster = cf_connected_graph.community_edge_betweenness()
#edge_bet = cf_cluster.as_clustering()
#cf_cluster.modularity

In [43]:
degrees = cf_connected_graph.degree()
plt.hist(degrees, bins=30)
plt.title("Degree Distribution")
plt.xlabel("Degree")
plt.ylabel("Frequency")
plt.show()

# Find the statistical significance of the CF network

In [44]:
"""iterations = 1000
random_size = np.zeros(iterations)
for i in range(iterations):
    random_set = np.random.choice(range(len(graph_main.vs)),len(cf_connected_graph.vs),replace=False) # select a random set of nodes from the original graph
    random_net = graph_main.induced_subgraph(random_set) # from graph_main, induce a subgraph with the randomly selected nodes
    random_net_comp = random_net.components() # find the components of the induced subgraph
    random_size[i] = len(random_net_comp[which_max(random_net)]) # select the largest component
"""

'iterations = 1000\nrandom_size = np.zeros(iterations)\nfor i in range(iterations):\n    random_set = np.random.choice(range(len(graph_main.vs)),len(cf_connected_graph.vs),replace=False) # select a random set of nodes from the original graph\n    random_net = graph_main.induced_subgraph(random_set) # from graph_main, induce a subgraph with the randomly selected nodes\n    random_net_comp = random_net.components() # find the components of the induced subgraph\n    random_size[i] = len(random_net_comp[which_max(random_net)]) # select the largest component\n'

In [45]:
"""fig = go.Figure(data=[go.Histogram(x=random_size,
                                   nbinsx=50,
                                   marker=dict(line=dict(color="black",width=1.5)))])

fig.update_layout(title='Biggest component size distribution of random networks',
    yaxis_title='Frequencies',
    xaxis_title='Random network sizes',
    width=800,  # Width in pixels
    height=600,  # Height in pixels
    xaxis=dict(range=[450,800]),
    
)

cf_bigcomp = len(cf_connected_graph.components()[which_max(cf_connected_graph)])
p_val = np.sum(random_size > cf_bigcomp)/iterations

fig.add_shape(
    type="line",
    x0=cf_bigcomp, x1=cf_bigcomp,
    y0=0, y1=120,  # vertical span
    line=dict(color="red", width=2, dash="dash")  # style
)
fig.show()

print(f"The P_val for the CF-related network is {p_val}")
print(cf_bigcomp)"""

'fig = go.Figure(data=[go.Histogram(x=random_size,\n                                   nbinsx=50,\n                                   marker=dict(line=dict(color="black",width=1.5)))])\n\nfig.update_layout(title=\'Biggest component size distribution of random networks\',\n    yaxis_title=\'Frequencies\',\n    xaxis_title=\'Random network sizes\',\n    width=800,  # Width in pixels\n    height=600,  # Height in pixels\n    xaxis=dict(range=[450,800]),\n    \n)\n\ncf_bigcomp = len(cf_connected_graph.components()[which_max(cf_connected_graph)])\np_val = np.sum(random_size > cf_bigcomp)/iterations\n\nfig.add_shape(\n    type="line",\n    x0=cf_bigcomp, x1=cf_bigcomp,\n    y0=0, y1=120,  # vertical span\n    line=dict(color="red", width=2, dash="dash")  # style\n)\nfig.show()\n\nprint(f"The P_val for the CF-related network is {p_val}")\nprint(cf_bigcomp)'

# Find the statistical significance of the amount of interactions in the CF-related network

In [46]:
"""cf_interactions = cf_graph.ecount()
print(f"The CF-related network has {cf_interactions} interactions between its constitutive proteins")
iterations = 1000
random_interaction = np.zeros(iterations)
for i in range(iterations):
    random_set = np.random.choice(int(graph_main.vcount()),int(cf_graph.vcount()),replace=False)
    random_net = graph_main.induced_subgraph(random_set)
    random_interaction[i] = random_net.ecount()

fig = go.Figure(data=[go.Histogram(x=random_interaction,nbinsx=50,marker=dict(line=dict(color="black",width=1.5)))])
fig.update_layout(title='Random network edges count distribution',
    yaxis_title='Frequencies',
    xaxis_title='Random network edges count',
    width=800,  # Width in pixels
    height=600,  # Height in pixels
    xaxis=dict(range=[700,4200]),
)

fig.add_shape(
    type="line",
    x0=cf_interactions, x1=cf_interactions,
    y0=0, y1=80,  # vertical span
    line=dict(color="red", width=2, dash="dash")  # style
)
fig.show()


cf_ecount = cf_graph.ecount()
p_value = np.sum(random_interaction > cf_ecount)/iterations
print(p_value)"""

'cf_interactions = cf_graph.ecount()\nprint(f"The CF-related network has {cf_interactions} interactions between its constitutive proteins")\niterations = 1000\nrandom_interaction = np.zeros(iterations)\nfor i in range(iterations):\n    random_set = np.random.choice(int(graph_main.vcount()),int(cf_graph.vcount()),replace=False)\n    random_net = graph_main.induced_subgraph(random_set)\n    random_interaction[i] = random_net.ecount()\n\nfig = go.Figure(data=[go.Histogram(x=random_interaction,nbinsx=50,marker=dict(line=dict(color="black",width=1.5)))])\nfig.update_layout(title=\'Random network edges count distribution\',\n    yaxis_title=\'Frequencies\',\n    xaxis_title=\'Random network edges count\',\n    width=800,  # Width in pixels\n    height=600,  # Height in pixels\n    xaxis=dict(range=[700,4200]),\n)\n\nfig.add_shape(\n    type="line",\n    x0=cf_interactions, x1=cf_interactions,\n    y0=0, y1=80,  # vertical span\n    line=dict(color="red", width=2, dash="dash")  # style\n)\nfi

# Find the significance of the average distance of the interactions within the CF network

In [47]:
"""avg_dist = np.matrix(graph_main.distances(source=cf_vertex,target=cf_vertex)).mean()
print(f"The average distance between pair of nodes in the CF-related network is {round(avg_dist,5)}")

iterations = 1000
random_avgdis = np.zeros(iterations)
for i in tqdm(range(iterations)):
    random_set = np.random.choice(int(graph_main.vcount()),int(cf_graph.vcount()),replace=False)
    random_distance = np.matrix(graph_main.distances(source=random_set,target=random_set)) # retrieve the matrix containing the distances between each pair of the randomly selected nodes
    random_avgdis[i] = random_distance.mean() # get the mean distance between pair of nodes

fig = go.Figure(data=[go.Histogram(x=random_avgdis,nbinsx=50,marker=dict(line=dict(color="black",width=1.5)))])
fig.update_layout(title='Random network average distance distribution',
    yaxis_title='Frequencies',
    xaxis_title='Random network average distance',
    width=800,  # Width in pixels
    height=600,  # Height in pixels
    xaxis=dict(range=[2.55,2.835]),
)

fig.add_shape(
    type="line",
    x0=avg_dist, x1=avg_dist,
    y0=0, y1=110,  # vertical span
    line=dict(color="red", width=2, dash="dash")  # style
)

fig.show()"""

'avg_dist = np.matrix(graph_main.distances(source=cf_vertex,target=cf_vertex)).mean()\nprint(f"The average distance between pair of nodes in the CF-related network is {round(avg_dist,5)}")\n\niterations = 1000\nrandom_avgdis = np.zeros(iterations)\nfor i in tqdm(range(iterations)):\n    random_set = np.random.choice(int(graph_main.vcount()),int(cf_graph.vcount()),replace=False)\n    random_distance = np.matrix(graph_main.distances(source=random_set,target=random_set)) # retrieve the matrix containing the distances between each pair of the randomly selected nodes\n    random_avgdis[i] = random_distance.mean() # get the mean distance between pair of nodes\n\nfig = go.Figure(data=[go.Histogram(x=random_avgdis,nbinsx=50,marker=dict(line=dict(color="black",width=1.5)))])\nfig.update_layout(title=\'Random network average distance distribution\',\n    yaxis_title=\'Frequencies\',\n    xaxis_title=\'Random network average distance\',\n    width=800,  # Width in pixels\n    height=600,  # Heigh

In [48]:
"""p_val = np.sum(random_avgdis < avg_dist)/iterations # calculate the p-value for the average distance of the CF-related network

print(p_val)"""

'p_val = np.sum(random_avgdis < avg_dist)/iterations # calculate the p-value for the average distance of the CF-related network\n\nprint(p_val)'

In [49]:
"""z_avgdis = (avg_dist - random_avgdis.mean())/random_avgdis.std()
print(f"The quantile is {z_avgdis}")

print(random_avgdis[random_avgdis < avg_dist].sum())"""

'z_avgdis = (avg_dist - random_avgdis.mean())/random_avgdis.std()\nprint(f"The quantile is {z_avgdis}")\n\nprint(random_avgdis[random_avgdis < avg_dist].sum())'

# ddPPR algorithm

## PM results from co-IP interactome (prof. Paulo Matos)

In [50]:
confidence = 4

wt_interactors = pd.read_excel("PM_wt_membrane_interactors.xlsx")
df_interactors = pd.read_excel("PM_df_membrane_interactors.xlsx")

wt_confident_interactors = wt_interactors.loc[wt_interactors["CCS"] >= confidence]["Uniprot_ID"].to_list() #[:20]
df_confident_interactors = df_interactors.loc[df_interactors["CCS"] >= confidence]["Uniprot_ID"].to_list() #[:20]

print(len(wt_confident_interactors))
print(len(df_confident_interactors))

56
225


In [51]:
added_int = add_interactors(graph=graph_main,
                subgraph=cf_connected_graph,
                int_list=unique(wt_confident_interactors+df_confident_interactors),
                attribute="name",
                show_statistic=False)

15 experimental interactors have no interaction described in the current model


Q15582 has no neighbor in the selected subgraph
Q14331 has no neighbor in the selected subgraph
P59190 has no neighbor in the selected subgraph
Q9Y421 has no neighbor in the selected subgraph
Q6EEV6 has no neighbor in the selected subgraph
Q2VIR3 has no neighbor in the selected subgraph
220 nodes were added


In [52]:
node_intersect_wt = intersect(wt_confident_interactors,cf_connected_graph.vs["name"])
if cftr in node_intersect_wt:
    node_intersect_wt.remove(cftr)
node_intersect_df = intersect(df_confident_interactors,cf_connected_graph.vs["name"])
if cftr in node_intersect_df:
    node_intersect_df.remove(cftr)

membint_index = which_attribute(node_intersect_wt,cf_connected_graph,"name") # get the indexes of the proteins that are in both networks
membint_index_df = which_attribute(node_intersect_df,cf_connected_graph,"name") 
seed_set = union(membint_index,membint_index_df)
seed_set = get_seeds(seed_set,cf_connected_graph,90)
membint_index = intersect(membint_index,seed_set)
membint_index_df = intersect(membint_index_df,seed_set)

membint_score = wt_interactors.loc[wt_interactors["Uniprot_ID"].isin(which_index(membint_index,cf_connected_graph,"name"))]["CCS"]
membint_score_df = df_interactors.loc[df_interactors["Uniprot_ID"].isin(which_index(membint_index_df,cf_connected_graph,"name"))]["CCS"]

# Add experimental interactors to CF-specific graph

print(f"There are {len(membint_index)} WT-CFTR membrane interactors in the connected CF-related network")
print(f"There are {len(membint_index_df)} DF-CFTR membrane interactors in the connected CF-related network")

cftr_index = cf_connected_graph.vs.find(name=cftr).index
# print('\n', node_intersect_wt,cftr)

There are 43 WT-CFTR membrane interactors in the connected CF-related network
There are 189 DF-CFTR membrane interactors in the connected CF-related network


In [53]:
control_seeds={
    "seed":membint_index + [cftr_index], # <--- change seed set here!
    "score":np.append(membint_score,5)
}

test_seeds={
    "seed":membint_index_df + [cftr_index], # <--- change seed set here!
    "score":np.append(membint_score_df,5)
}

all_seeds = union(control_seeds["seed"], test_seeds["seed"])
cent_measure_c = np.array(cf_connected_graph.betweenness())[control_seeds["seed"]]
cent_measure_t = np.array(cf_connected_graph.betweenness())[test_seeds["seed"]]
fig, ax = plt.subplots(1,2,figsize=(7,5))
ax[0].boxplot(cent_measure_c)
ax[1].boxplot(cent_measure_t)
plt.show()

In [54]:
print(cf_connected_graph.summary())

IGRAPH UN-- 1033 6095 -- 
+ attr: name (v)


## ddPPR calculation

In [55]:
def add_pathway_annotation(graph,pathway_df,graph_field:str,df_gene_field,df_path_field:str,inplace=False,voice=False):
    annotated_nodes = {}
    for node in graph.vs:
        if node[graph_field] in pathway_df[df_gene_field].to_list():
            annotated_nodes[node.index] = pathway_df.loc[pathway_df[df_gene_field]==node[graph_field],df_path_field].to_list()[0]
        else:
            continue
    if inplace:
        graph.vs[annotated_nodes.keys()][df_path_field] = [name for name in annotated_nodes.values()] # in-place modification of graph fields
    
    print(f"{len(annotated_nodes.values())} genes were annotated")
    return annotated_nodes

def ddPPR(g:ig.Graph,ctrl_dict,test_dict,background,directed,iterations:int,alpha:float,gamma:float,attribute_display:str,pathway=False,path_df=pd.DataFrame(),graph_path_field=""):

    seed_set_c = ctrl_dict["seed"]
    seed_set_t = test_dict["seed"]
    
    seeds = seed_set_c + seed_set_t
    node_ids = g.vs[attribute_display] # get the node IDs from the graph

    # Initiate control and test restart vectors
    restart_vector_c = np.full(
        g.vcount(),
        background
    )
    restart_vector_t = np.full(
        g.vcount(),
        background
    )

    # Load the seed restart weights in both vectors
    if seed_set_c:
        restart_vector_c[seed_set_c] = ctrl_dict["score"] # attribute membrane seeds full restart p
    if seed_set_t:   
        restart_vector_t[seed_set_t] = test_dict["score"] # attribute non-membrane seeds lower restart p 
        
    print(f"The control non-normalized restart vector entries sum to {restart_vector_c.sum()}")
    print(f"The test non-normalized restart vector entries sum to {restart_vector_t.sum()}")

    #norm = max(restart_vector_c.sum(),restart_vector_t.sum())
    restart_vector_c /= restart_vector_c.sum() # Normalize to sum to 1
    restart_vector_t /= restart_vector_t.sum()
    
    # Run the PPR algorithm
    es_pagerank_scores_c = np.array( # convert to np array for flexibility
        g.personalized_pagerank( # personalized page rank
        reset=restart_vector_c,
        damping=alpha, # damping probability
        directed=directed))
    es_pagerank_scores_t = np.array( 
        g.personalized_pagerank( 
        reset=restart_vector_t,
        damping=alpha,
        directed=directed))

    # Run a control PPR with unbiased restart probabilities
    ctrl_restart_vector = np.full(g.vcount(),
                              background) # restart vector for the random walk

    ctrl_pagerank_scores = np.array(g.personalized_pagerank(reset=ctrl_restart_vector, damping=alpha, directed=directed))

    dppr_c = (es_pagerank_scores_c - ctrl_pagerank_scores) # calculate the difference between the two pagerank scores
    dppr_t = (es_pagerank_scores_t - ctrl_pagerank_scores)
    ddppr = (dppr_t - dppr_c) / np.abs(dppr_c)
    
    # Perform pathway analysis 
    if pathway:
        path_df["Which_genes"] = pd.Series([[] for _ in range(len(path_df))]) # initialize a column to store the genes in each pathway
        path_ddppr = np.zeros(len(path_df))
        for node in g.vs:
            pathways = node[graph_path_field] # fetch the pathways in which the node is included
            pathways = [pathways] if not isinstance(pathways,list) else pathways # check whether it is a list of pathways
            annotation = len(pathways)            
            pathways_with_node = path_df[graph_path_field].isin(pathways) # mask to retireve the associated pathways
            path_ddppr[pathways_with_node] += ddppr[node.index] # add the score per pathway
            for idx in path_df.index[pathways_with_node]:
                path_df.at[idx,"Which_genes"].append(node_ids[node.index]) # add the node ID to the pathway genes list

    # Run multiple network rewirings to calculate significance scores

    null_ddppr_scores = np.zeros((g.vcount(), iterations)) # Matrix initialization to collect null difference scores from the rewired graph
    null_path_ddppr_scores = np.zeros((len(path_df),iterations))
    for i in tqdm(range(iterations)):
        # Rewire the graph while preserving node degrees
        rewired_graph = g.copy()
        rewired_graph.rewire(mode="simple", n=10 * rewired_graph.ecount())
        
        # Run PPR on the rewired graph using the same restart vector
        null_ppr_c = np.array(rewired_graph.personalized_pagerank(reset=restart_vector_c, damping=alpha, directed=False))
        null_ppr_t = np.array(rewired_graph.personalized_pagerank(reset=restart_vector_t, damping=alpha, directed=False))
        null_ctrl_ppr = np.array(rewired_graph.personalized_pagerank(reset=ctrl_restart_vector, damping=alpha, directed=False))
        null_dppr_c = (null_ppr_c - null_ctrl_ppr) 
        null_dppr_t = (null_ppr_t - null_ctrl_ppr) 
        null_ddppr = (null_dppr_t - null_dppr_c) / np.abs(null_dppr_c)
        null_ddppr_scores[:, i] = null_ddppr

        # perform pathway enrichment analysis significance evaluation
        if pathway:
            null_path_ddppr = np.zeros(len(path_df))
            for node in g.vs:
                pathways = node[graph_path_field] # fetch the pathways in which the node is included
                pathways = [pathways] if not isinstance(pathways,list) else pathways # check whether it is a list of pathways
                annotation = len(pathways)
                pathways_with_node = path_df[graph_path_field].isin(pathways)
                null_path_ddppr[pathways_with_node] += null_ddppr[node.index]
            null_path_ddppr_scores[:,i] = null_path_ddppr

    # Compute z-scores and p-values
    p_val_two_tailed = np.sum(np.abs(null_ddppr_scores) >= np.abs(ddppr[:,np.newaxis]),axis=1)/iterations
    _, fdr_corrected_pvalue, _, _ = multipletests(p_val_two_tailed, method='fdr_bh')

    if pathway:
        # Compute z-scores and p-values for pathway enrichment
        path_p_val_two_tailed = np.sum(np.abs(null_path_ddppr_scores) >= np.abs(path_ddppr[:,np.newaxis]),axis=1)/iterations
        _, path_fdr_corrected_pvalue, _, _ = multipletests(path_p_val_two_tailed, method='fdr_bh')   
        path_df["Enrichment"] = path_ddppr
        path_df["P_value"] = path_p_val_two_tailed
        path_df["FDR"] = path_fdr_corrected_pvalue

    final_score = pd.DataFrame(data={ # create a df with the scores and p-values
        "UniprotID":node_ids,
        "Control_PPR":es_pagerank_scores_c,
        "Test_PPR":es_pagerank_scores_t,
        "dPPR_C":dppr_c,
        "dPPR_t":dppr_t,
        "ddPPR":ddppr,
        "P_value_2t":p_val_two_tailed,
        "FDR":fdr_corrected_pvalue
    })
    
    final_score["Is seed"] = 0
    for node in final_score.index:
        if node in seed_set_c and node not in seed_set_t:
            final_score.loc[node,"Is seed"] = "Control"
        elif node in seed_set_t and node not in seed_set_c:
            final_score.loc[node,"Is seed"] = "Test"
        elif node in seed_set_c and node in seed_set_t:
            final_score.loc[node,"Is seed"] = "Control + Test"
        else:
            final_score.loc[node,"Is seed"] = "Nonseed"
    #final_score["Is seed"] = final_score.index.isin(seeds) # add a column to indicate if the node is a seed or not
    
    if pathway:
        return {"ddPPR":final_score, "Null Score":null_ddppr_scores,"path_ddPPR":path_df} 
    
    return {"ddPPR":final_score, "Null Score":null_ddppr_scores}

# shuffle seeds within a centrality margin similar to the original seeds

In [56]:
"""
    z_scores = (ddppr  - np.mean(null_ddppr_scores, axis=1)) / np.std(null_ddppr_scores, axis=1)
    p_values = 2*(1 - norm.cdf(np.abs(z_scores)))
    _, fdr_corrected_pvals, _, _ = multipletests(p_values, method='fdr_bh') # Multiple testing correction
    """

"\n    z_scores = (ddppr  - np.mean(null_ddppr_scores, axis=1)) / np.std(null_ddppr_scores, axis=1)\n    p_values = 2*(1 - norm.cdf(np.abs(z_scores)))\n    _, fdr_corrected_pvals, _, _ = multipletests(p_values, method='fdr_bh') # Multiple testing correction\n    "

In [58]:
ctrl_seeds = len(control_seeds["score"])
t_seeds = len(test_seeds["score"])
print(f"{ctrl_seeds} WT seeds and {t_seeds} F508del seeds")

"""dict_wt_df_ddppr = ddPPR(
    g=cf_connected_graph,
    ctrl_dict=control_seeds,
    test_dict=test_seeds,
    background=1e-12,
    directed=False,
    iterations=10000,
    alpha=0.45,
    gamma=1,
    attribute_display="name",
)"""

#is_seed = dict_wt_df_ddppr["ddPPR"]["Is seed"]

44 WT seeds and 190 F508del seeds


'dict_wt_df_ddppr = ddPPR(\n    g=cf_connected_graph,\n    ctrl_dict=control_seeds,\n    test_dict=test_seeds,\n    background=1e-12,\n    directed=False,\n    iterations=10000,\n    alpha=0.45,\n    gamma=1,\n    attribute_display="name",\n)'

In [165]:
def uniprot_to_name(map_df,uniprot_list,name_column):
    name_list = []
    for protein in uniprot_list:
        if protein in map_df.iloc[:,0]:
            name = map_df.loc[protein,name_column]
            name_list.append(name)
        else:
            name_list.append("")
    return name_list
map = pd.read_csv("UniprotID_Genename_mapping.csv").iloc[:,1:].set_index("UniprotID")
cf_connected_graph.vs["gene_name"] = uniprot_to_name(map,cf_connected_graph.vs["name"],"Gene Name") # add the gene name to the cf graph

"""for protein in wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"]<=0.05) & (wt_df_ddppr["ddPPR"] < 0)]["UniprotID"].to_list():
    print(protein)"""
#wt_df_ddppr = dict_wt_df_ddppr["ddPPR"]
#wt_df_ddppr["Gene Name"] = cf_connected_graph.vs["gene_name"]
wt_df_ddppr = pd.read_csv("ddPPR_wt_df_PauloMatos_seedset_results.csv").iloc[:,1:]
seed_indexing = ["Control", "Test", "Control + Test"]
#wt_df_ddppr.to_csv("ddPPR_wt_df_PauloMatos_seedset_results.csv")

wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"] <=0.05) ]

,UniprotID,Control_PPR,Test_PPR,dPPR_C,dPPR_t,ddPPR,P_value_2t,FDR,Is seed,Gene Name,Centrality,Betweenness
17,P10415,0.001371,0.000341,6.253495e-05,-0.000967,-16.466690,0.0147,0.799216,Nonseed,BCL2,1733.396359,1733.396359
53,P45983,0.001385,0.000618,-9.113581e-05,-0.000858,-8.416788,0.0341,0.996705,Nonseed,MAPK8,2117.614960,2117.614960
62,P22303,0.000333,0.000034,-2.450904e-04,-0.000545,-1.222406,0.0472,0.999800,Nonseed,ACHE,0.555129,0.555129
84,O14745,0.016230,0.003522,1.506621e-02,0.002358,-0.843477,0.0091,0.778723,Control + Test,SLC9A3R1,1457.803557,1457.803557
105,Q9UBN7,0.001507,0.001053,-5.268194e-05,-0.000507,-8.629396,0.0350,0.996705,Nonseed,HDAC6,2790.675183,2790.675183
185,Q01970,0.000808,0.000341,5.756933e-05,-0.000410,-8.122297,0.0125,0.799216,Nonseed,PLCB3,41.976278,41.976278
200,P01579,0.000645,0.000010,1.345529e-05,-0.000621,-47.148958,0.0008,0.275467,Nonseed,IFNG,6.635893,6.635893
221,P11021,0.002705,0.001652,-1.402381e-04,-0.001193,-7.505588,0.0391,0.999800,Nonseed,HSPA5,10982.179162,10982.179162
240,Q9ULV1,0.000774,0.000214,1.197532e-04,-0.000440,-4.671338,0.0256,0.996705,Nonseed,FZD4,63.666566,63.666566
251,Q9Y3M8,0.000982,0.000050,3.534807e-04,-0.000578,-2.635335,0.0328,0.996705,Nonseed,STARD13,19.247435,19.247435


In [132]:
f, ax = plt.subplots(1,1,figsize=(7,5))
results = wt_df_ddppr.copy()
results.sort_values(by="ddPPR", ascending=False, inplace=True)
results.index = range(len(results)) # reset index to have a continuous index
ax.plot(results.index.to_list(),results["ddPPR"],label="Control PPR") # 
ax.scatter(results.loc[results["P_value_2t"]<=0.05].index.to_list(),results.loc[results["P_value_2t"]<=0.05]["ddPPR"],c="yellow",alpha=0.7,s=20) # .index.to_list()
plt.show()

results

,UniprotID,Control_PPR,Test_PPR,dPPR_C,dPPR_t,ddPPR,P_value_2t,FDR,Is seed,Gene Name,Centrality,Betweenness
0,P27635,0.001369,0.004904,2.578784e-06,0.003538,1370.807860,0.0012,0.309900,Test,RPL10,912.613093,912.613093
1,P51665,0.000787,0.003785,-4.417448e-06,0.002994,678.746264,0.0017,0.351220,Test,PSMD7,64.821612,64.821612
2,P50991,0.000973,0.003343,-7.047159e-06,0.002363,336.335219,0.0021,0.361550,Test,CCT4,483.042089,483.042089
3,P07384,0.001223,0.003928,-1.194353e-05,0.002693,226.506666,0.0070,0.778723,Test,CAPN1,1648.952166,1648.952166
4,P35998,0.000887,0.003868,-2.916788e-05,0.002952,102.212764,0.0135,0.799216,Test,PSMC2,304.929524,304.929524
...,...,...,...,...,...,...,...,...,...,...,...,...
1028,Q9UMR2,0.001719,0.000459,6.363014e-05,-0.001195,-19.787911,0.0127,0.799216,Nonseed,DDX19B,2500.128683,2500.128683
1029,P20810,0.000817,0.000211,2.591720e-05,-0.000580,-23.389836,0.0063,0.778723,Nonseed,CAST,354.015840,354.015840
1030,P01579,0.000645,0.000010,1.345529e-05,-0.000621,-47.148958,0.0008,0.275467,Nonseed,IFNG,6.635893,6.635893
1031,O94817,0.000666,0.000160,4.320876e-06,-0.000502,-117.295525,0.0007,0.275467,Nonseed,ATG12,31.102321,31.102321


In [182]:
import plotly.express as px
import pandas as pd

# Sample structure (replace with your actual data)

# Plot
fig = px.bar(
    wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"] <= 0.05) & (np.abs(wt_df_ddppr["ddPPR"]) > 0) ].sort_values("ddPPR"), #& (wt_df_ddppr["Is seed"] == False)
    x="Gene Name",
    y="ddPPR",
    color="P_value_2t",
    color_continuous_scale="RdBu_r",  # red=low p-value, blue=high
    title="ddPPR WT vs. F508del membrane seed sets"
)

fig.update_layout(
    yaxis_title="ddPPR Score",
    xaxis_title="Node",
    bargap=0.05,
    coloraxis_colorbar=dict(title="p-value"),
    width=800,  # Width in pixels
    height=600  # Height in pixels
)

fig.show()

# check if proteins are close to the CFTR!!
#biogrid and HURI
#directed: omnipath
# reactome for pathway enrichment

#### Statistical Summary

In [ ]:
fig = go.Figure(data=go.Scatter(x=wt_df_ddppr.loc[wt_df_ddppr["Is seed"]=="Nonseed"]["P_value_2t"],
                                y=wt_df_ddppr.loc[wt_df_ddppr["Is seed"]=="Nonseed"]["ddPPR"],
                                mode='markers',
                                hovertext=wt_df_ddppr.loc[wt_df_ddppr["Is seed"]=="Nonseed"]["Gene Name"],
                                name="Non-seed nodes"))

fig.update_layout(title='PPR score vs P value',
    yaxis_title='ddPPR Score (relevance)',
    xaxis_title='P value',
    width=800,  # Width in pixels
    height=600  # Height in pixels
)

fig.add_trace(go.Scatter(
    x=wt_df_ddppr.loc[~(wt_df_ddppr["Is seed"]=="Nonseed") & (~wt_df_ddppr["UniprotID"].isin([cftr]))]["P_value_2t"],
    y=wt_df_ddppr.loc[~(wt_df_ddppr["Is seed"]=="Nonseed") & (~wt_df_ddppr["UniprotID"].isin([cftr]))]["ddPPR"],
    mode='markers',
    name='Seed nodes',
    marker=dict(color='green', size=7, symbol='square'),
    hovertext=wt_df_ddppr.loc[~(wt_df_ddppr["Is seed"]=="Nonseed") & (~wt_df_ddppr["UniprotID"].isin([cftr]))]["Gene Name"]
))

fig.add_trace(go.Scatter(
    x=wt_df_ddppr.loc[wt_df_ddppr["UniprotID"] == cftr]["P_value_2t"],
    y=wt_df_ddppr.loc[wt_df_ddppr["UniprotID"] == cftr]["ddPPR"],
    mode='markers',
    name='CFTR',
    marker=dict(color='red', size=10, symbol='star'),
    hovertext=wt_df_ddppr.loc[wt_df_ddppr["UniprotID"] == cftr]["Gene Name"].values[0]
))

fig.add_shape(
    type='line',
    x0=0.05, x1=0.05,
    y0=0, y1=1,  # In yref="paper", 0–1 spans the full y-axis height
    line=dict(color='Red', dash='dash'),
    xref='x',
    yref='paper'
)

fig.add_shape(
    type='line',
    x0=0.0, x1=1,
    y0=0, y1=0,  # In yref="paper", 0–1 spans the full y-axis height
    line=dict(color='Black', dash='dash'),
    xref='paper',
    yref='y'
)

fig.show()


In [ ]:
score = "ddPPR"

fig = go.Figure(data=go.Scatter(y=wt_df_ddppr.loc[(wt_df_ddppr["Betweenness"]>0) & (wt_df_ddppr["Is seed"] == "Nonseed")][score],
                                x=np.log10(wt_df_ddppr.loc[(wt_df_ddppr["Betweenness"]>0) & (wt_df_ddppr["Is seed"] == "Nonseed")]["Betweenness"]),
                                mode='markers',
                                hovertext=wt_df_ddppr.loc[(wt_df_ddppr["Betweenness"]>0) & (wt_df_ddppr["Is seed"] == "Nonseed") ]["Gene Name"],
                                name="All nodes"))

# Update figure's layout
fig.update_layout(title='PPR score vs Betweeness',
    yaxis_title=f'{score} Score (relevance)',
    xaxis_title='Betweenes (log10 scale)',
    width=800,  # Width in pixels
    height=600  # Height in pixels
)

# Add a trace to identify significant (p_val <= 0.05) seed nodes
fig.add_trace(go.Scatter(
    y=wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"] <= 0.05) & (wt_df_ddppr["Is seed"].isin(seed_indexing))][score],
    x=np.log10(wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"] <= 0.05) & (wt_df_ddppr["Is seed"].isin(seed_indexing))]["Betweenness"]),
    mode='markers',
    name='Significant seed nodes',
    marker=dict(color='cadetblue', size=7, symbol='circle'),
    hovertext=[f"{id} - {p_val}" for id,p_val in zip(wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"] <= 0.05) & (wt_df_ddppr["Is seed"].isin(seed_indexing))]["Gene Name"],wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"] <= 0.05) & (wt_df_ddppr["Is seed"].isin(seed_indexing))]["P_value_2t"])]
))

# Add a trace to identify non-significant seed nodes
fig.add_trace(go.Scatter(
    y=wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"] > 0.05) & (wt_df_ddppr["Is seed"].isin(seed_indexing))][score],
    x=np.log10(wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"] > 0.05) & (wt_df_ddppr["Is seed"].isin(seed_indexing))]["Betweenness"]),
    mode='markers',
    name='Non-significant nodes',
    marker=dict(color='wheat', size=7, symbol='circle'),
    hovertext=[f"{id} - {p_val}" for id,p_val in zip(wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"] > 0.05) & (wt_df_ddppr["Is seed"].isin(seed_indexing))]["Gene Name"],wt_df_ddppr.loc[(wt_df_ddppr["P_value_2t"] > 0.05)& (wt_df_ddppr["Is seed"].isin(seed_indexing))]["P_value_2t"])]
))

# Add a trace to identify non-seed significant nodes
"""fig.add_trace(go.Scatter(
    y=final_score.loc[(final_score["P_value"] <= 0.05) & (final_score["Is seed"] == False)][score],
    x=np.log10(cf_centralities_measure.loc[(final_score["P_value"] <= 0.05) & (final_score["Is seed"] == False)]["Betweeness"]),
    mode='markers',
    name='Non-seed significant nodes',
    marker=dict(color='purple', size=7, symbol='circle'),
    hovertext=[f"{id} - {p_val}" for id,p_val in zip(non_seed_rel,final_score.loc[(final_score["P_value"] <= 0.05) & (final_score["Is seed"] == False)]["P_value"])]
))"""

"""fig = go.Figure(data=go.Scatter(y=final_score.loc[~final_score["UniprotID"].isin([name for name in memint_name]+neighbor_names+[cftr])]["PageRank"],
                                x=np.log10(cf_centralities_measure.loc[~cf_centralities_measure["UniprotID"].isin([name for name in memint_name]+neighbor_names+[cftr])]["Betweeness"]),
                                mode='markers',
                                hovertext=cf_connected_graph.vs["name"],
                                name="All nodes"))
fig.update_layout(title='PPR score vs Betweeness',
    yaxis_title='Page Rank Score (relevance)',
    xaxis_title='Betweenes (log10 scale)',
    width=800,  # Width in pixels
    height=600  # Height in pixels
)

fig.add_trace(go.Scatter(
    y=final_score.loc[final_score["UniprotID"].isin(cf_neighbors)]["PageRank"],
    x=np.log10(cf_centralities_measure.loc[cf_centralities_measure["UniprotID"].isin(cf_neighbors)]["Betweeness"]),#final_score.loc[final_score["UniprotID"].isin(cf_neighbors)]["PageRank"],
    mode='markers',
    name='Membrane interactors',
    marker=dict(color='purple', size=7, symbol='square'),
    hovertext=cf_neighbors
))
"""

fig.show()

c:\Users\jpedr\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

divide by zero encountered in log10



In [ ]:
len(cf_connected_graph.neighborhood(cftr_index))

105

In [ ]:
print(f"Node dPPR distribution over log10(Betweeness) Centrality")

final_score_noseeds = wt_df_ddppr#.loc[wt_df_ddppr["Is seed"] == True]
centrality_score_g = np.array(cf_connected_graph.betweenness())+1
print(len(final_score_noseeds))
ppr0_1 = final_score_noseeds.loc[(np.log10(centrality_score_g)>=0) & (np.log10(centrality_score_g)<1)]

ppr1_2 = final_score_noseeds.loc[(np.log10(centrality_score_g)>=1) & (np.log10(centrality_score_g)<2)]

ppr2_3 = final_score_noseeds.loc[(np.log10(centrality_score_g)>=2) & (np.log10(centrality_score_g)<3)]

ppr3_4 = final_score_noseeds.loc[(np.log10(centrality_score_g)>=3) & (np.log10(centrality_score_g)<4)]

f, ax = plt.subplots(1,1,figsize=(7,5))

ax.boxplot([ppr0_1["ddPPR"],ppr1_2["ddPPR"],ppr2_3["ddPPR"],ppr3_4["ddPPR"]],labels=["0-1","1-2","2-3","3->4"])
ax.set_title("dPPR score distribution by log10(Betweeness) Centrality")
plt.show()

Node dPPR distribution over log10(Betweeness) Centrality
1033


C:\Users\jpedr\AppData\Local\Temp\ipykernel_20000\3941352651.py:16: MatplotlibDeprecationWarning:

The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.



## Reactome Pathway Enrichment

In [162]:
import pandas as pd
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
from scipy.stats import hypergeom

def reshape_reactome_df(reactome_df, pathway_col='GeneSet', genes_col='Genes', description_col='Description'):
    rows = []
    for _, row in reactome_df.iterrows():
        pathway = row[pathway_col]
        description = row[description_col]
        # Split the genes by tab, comma, or whitespace as needed
        genes = str(row[genes_col]).strip().split(' ')
        for gene in genes:
            gene = gene.strip()
            if gene:
                rows.append({'GeneSet': pathway, 'Description': description, 'Genes': gene})
    return pd.DataFrame(rows)

def read_gmt_to_df(filepath):
    records = {"GeneSet" : [],
               "Description" : [],
               "Genes" : []}
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            gene_set = parts[0]
            description = parts[1]
            genes = "   ".join(parts[2:])
            #records.append([gene_set, description, gene])
            records["GeneSet"].append(gene_set)
            records["Description"].append(description)
            records["Genes"].append(genes)
    return pd.DataFrame(records)

def gene_set_hypergeometric_enrichment(input_genes, gmt_df, background_genes=None, min_genes=10, max_genes=40, fdr=0.05):
    input_genes = set(input_genes)
    gmt_df = reshape_reactome_df(gmt_df)
    
    if background_genes is None:
        background_genes = set(gmt_df['Genes'].unique())
    else:
        background_genes = set(background_genes)
    
    results = {'GeneSet': [],
            'SetSize': [],
            'InputOverlap': [],
            'P-value': [],
            'Genes': []}
    M = len(background_genes)  # total number of background genes
    N = len(input_genes & background_genes)  # number of input genes in background

    # Group genes by gene set
    gene_sets = gmt_df.groupby('GeneSet')['Genes'].apply(set)
    for set_name, genes in gene_sets.items():
        genes_in_set = genes & background_genes  # only genes in background
        n = len(genes_in_set)  # number of genes in the gene set
        if n < min_genes or n > max_genes:
            continue

        x = len(input_genes & genes_in_set)  # overlap

        if x == 0:
            continue

        # Hypergeometric test: P(X >= x)
        p_value = hypergeom.sf(x - 1, M, n, N)

        results['GeneSet'].append(set_name)
        results['SetSize'].append(n)
        results['InputOverlap'].append(x)
        results['P-value'].append(p_value)
        results['Genes'].append(', '.join(input_genes & genes_in_set))

    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    if results_df.empty:
        return results_df

    # Multiple testing correction
    _, qvals, _, _ = multipletests(results_df['P-value'], method='fdr_bh')
    results_df['FDR'] = qvals

    # Filter and sort
    results_df = results_df[results_df['FDR'] <= fdr]
    results_df = results_df.sort_values('FDR')

    return results_df


In [163]:
# Example usage
reactome_db = read_gmt_to_df("ReactomePathways.gmt")
reactome_db

,GeneSet,Description,Genes
0,2-LTR circle formation,R-HSA-164843,BANF1 HMGA1 LIG4 PSIP1 XRCC4 XRCC5 ...
1,3-Methylcrotonyl-CoA carboxylase deficiency,R-HSA-9909438,MCCC1 MCCC2
2,3-hydroxyisobutyryl-CoA hydrolase deficiency,R-HSA-9916722,HIBCH
3,3-methylglutaconic aciduria,R-HSA-9914274,AUH
4,5-Phosphoribose 1-diphosphate biosynthesis,R-HSA-73843,PRPS1 PRPS1L1 PRPS2
...,...,...,...
2780,tamatinib-resistant FLT3 mutants,R-HSA-9703009,FLT3
2781,tandutinib-resistant FLT3 mutants,R-HSA-9702636,FLT3
2782,trans-Golgi Network Vesicle Budding,R-HSA-199992,ACBD3 AP1B1 AP1G1 AP1G2 AP1M1 AP1M2 ...
2783,vRNA Synthesis,R-HSA-192814,NP NS PA PARP1 PB1 PB2


In [166]:
pathways_per_gene = reshape_reactome_df(reactome_db)[["GeneSet","Genes"]].groupby("Genes")["GeneSet"].apply(list).reset_index()
pathways_per_gene["Genes"] = pathways_per_gene["Genes"].apply(lambda x: x.strip('()'))
#print(len(pathways_per_gene))
ppg_cf = pathways_per_gene.loc[pathways_per_gene["Genes"].isin(cf_connected_graph.vs["gene_name"])]

added_annotation = add_pathway_annotation(cf_connected_graph,pathways_per_gene,"gene_name","Genes","GeneSet",inplace=True)

877 genes were annotated


In [169]:
print(cf_connected_graph.summary())

IGRAPH UN-- 1033 6095 -- 
+ attr: GeneSet (v), gene_name (v), name (v)


In [170]:
"""enri_pathway_results = ddPPR(g=cf_connected_graph,
    ctrl_dict=control_seeds,
    test_dict=test_seeds,
    background=1e-12,
    directed=False,
    iterations=10000,
    alpha=0.45,
    gamma=1,
    attribute_display="name",
    pathway=True,
    path_df=reactome_db,
    graph_path_field="GeneSet")
"""
enri_pathway_results_df = pd.read_csv("10000_iterations_path_enrichment.csv")
enri_pathway_results_df = enri_pathway_results_df.iloc[:,1:]
#enri_pathway_results = enri_pathway_results["path_ddPPR"]

In [171]:
#enri_pathway_results_df = enri_pathway_results["path_ddPPR"]
enri_pathway_results_df.loc[(enri_pathway_results_df["P_value"] <= 0.05) & (enri_pathway_results_df["Enrichment"] < 0)]
#enri_pathway_results_df.to_csv("10000_iterations_path_enrichment.csv")

,GeneSet,Description,Genes,Enrichment,P_value,FDR,Which_genes
21,APC-Cdc20 mediated degradation of Nek2A,R-HSA-179409,ANAPC1 ANAPC10 ANAPC11 ANAPC15 ANAPC16...,-15.377947,0.0497,0.474022,"['P0CG48', 'P62979', 'P62987']"
23,APC/C:Cdc20 mediated degradation of Cyclin B,R-HSA-174048,ANAPC1 ANAPC10 ANAPC11 ANAPC15 ANAPC16...,-15.377947,0.0497,0.474022,"['P0CG48', 'P62979', 'P62987']"
36,ATF6B (ATF6-beta) activates chaperones,R-HSA-8874177,ATF6B HSPA5 MBTPS1 MBTPS2,-7.505588,0.0422,0.458769,['P11021']
61,Activated NTRK2 signals through FRS2 and FRS3,R-HSA-9028731,BDNF FRS2 FRS3 GRB2 HRAS KRAS NRAS...,-15.659068,0.0384,0.434732,"['P62993', 'P01116']"
65,Activated NTRK2 signals through RAS,R-HSA-9026519,BDNF GRB2 HRAS KRAS NRAS NTF4 NTRK...,-15.659068,0.0384,0.434732,"['P62993', 'P01116']"
...,...,...,...,...,...,...,...
2474,"TICAM1,TRAF6-dependent induction of TAK1 complex",R-HSA-9014325,MAP3K7 RPS27A TAB1 TAB2 TAB3 TICAM1 ...,-15.377947,0.0497,0.474022,"['P0CG48', 'P62979', 'P62987']"
2475,TICAM1-dependent activation of IRF3/IRF7,R-HSA-9013973,IKBKE IRF3 IRF7 OPTN RPS27A TANK T...,-15.377947,0.0497,0.474022,"['P0CG48', 'P62979', 'P62987']"
2530,The NLRP1 inflammasome,R-HSA-844455,BCL2 BCL2L1 NLRP1,-16.466690,0.0162,0.290997,['P10415']
2673,VLDLR internalisation and degradation,R-HSA-8866427,AP2A1 AP2A2 AP2B1 AP2M1 AP2S1 CLTA ...,-15.377947,0.0497,0.474022,"['P0CG48', 'P62979', 'P62987']"


In [183]:
significant_enri_path = enri_pathway_results_df.loc[(enri_pathway_results_df["P_value"] <= 0.1) & (enri_pathway_results_df["Enrichment"] > 0)]


### Similarity Analysis

In [184]:
import networkx as nx
import math
import pandas as pd
from itertools import combinations
from collections import defaultdict
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

def compute_information_content(G):
    """Compute IC for each node based on descendant count (inverse specificity)."""
    N = len(G)
    IC = {}
    for node in G.nodes:
        freq = len(nx.descendants(G, node)) + 1  # +1 to avoid log(0); retrieves how many nodes are accessible from a given node
        IC[node] = -math.log(freq / N) # info content is the log of how many nodes are reachable from that node
    return IC

def get_LCAs(G, node1, node2):
    """Return the set of lowest common ancestors of two nodes in a DAG."""
    anc1 = nx.ancestors(G, node1) | {node1}
    anc2 = nx.ancestors(G, node2) | {node2}
    return anc1 & anc2

def resnik_similarity(G, IC, node1, node2):
    """Compute Resnik similarity between two nodes."""
    LCAs = get_LCAs(G, node1, node2)
    if not LCAs:
        return 0.0
    # Get the highest LCA (the node that can reach the farthest in the network, the highest node in the hirearchy)
    return max(IC[a] for a in LCAs) # IC being a dictionary for all nodes in the graph

def compute_resnik_matrix(G, pathways, IC, normalize=True):
    """Compute a symmetric Resnik similarity matrix between pathways."""
    max_IC = max(IC.values()) if normalize else 1.0
    matrix = pd.DataFrame(index=pathways, columns=pathways, dtype=float)

    for a in pathways:
        for b in pathways:
            if pd.isna(matrix.loc[a, b]):
                sim = resnik_similarity(G, IC, a, b)
                if normalize and max_IC > 0:
                    sim /= max_IC
                matrix.loc[a, b] = sim
                matrix.loc[b, a] = sim  # symmetry
# Fill diagonal with self-similarity (should be highest possible value)
    for a in pathways:
        matrix.loc[a, a] = 1.0  # because sim(node, node) = max_IC / max_IC = 1.0 if normalized

    return matrix

def filter_redundant_pathways_by_resnik(G, enriched_pathways, similarity_threshold=0.3):
    """
    Given a DAG G and a list of enriched pathways (nodes in G),
    return a filtered list keeping only the most specific pathway per cluster
    based on Resnik similarity.
    
    Args:
        G (networkx.DiGraph): Reactome DAG.
        enriched_pathways (list): List of Reactome pathway IDs (R-HSA-XXXXX).
        similarity_threshold (float): Distance threshold for clustering (1 - similarity).
    
    Returns:
        filtered_pathways (list): Most specific (high IC) pathway per cluster.
        resnik_matrix (pd.DataFrame): Full similarity matrix (normalized).
    """
    # Step 1: Compute information content
    IC = compute_information_content(G)

    # Step 2: Compute Resnik similarity matrix
    resnik_matrix = compute_resnik_matrix(G=G, pathways=enriched_pathways, IC=IC, normalize=True)

    # Step 3: Cluster similar pathways using hierarchical clustering
    dist_matrix = 1 - resnik_matrix.fillna(0)
    np.fill_diagonal(dist_matrix.values, 0)  # Fix the diagonal
    condensed = squareform(dist_matrix.values)
    linkage_matrix = linkage(condensed, method='average') # the clustering process itself

    cluster_ids = fcluster(linkage_matrix, t=similarity_threshold, criterion='distance')
    # Step 4: For each cluster, keep the most specific (highest IC) term
    clusters = defaultdict(list)
    for pathway, cluster_id in zip(enriched_pathways, cluster_ids):
        clusters[cluster_id].append(pathway)

    filtered_pathways = []
    for group in clusters.values():
        most_specific = max(group, key=lambda p: IC[p])
        filtered_pathways.append(most_specific)

    return filtered_pathways, resnik_matrix

In [189]:
# Load the edge list
df = pd.read_csv("ReactomePathwaysRelation.tsv", sep="\t", header=None, names=["parent", "child"])

# Create a directed graph
G = nx.DiGraph()
G.add_edges_from(df.values)
assert nx.is_directed_acyclic_graph(G), "Graph must be a DAG!"

relevant_terms = significant_enri_path["Description"]
relevant_terms = [p for p in significant_enri_path["Description"] if p in G.nodes]

filtered_pathways, resnik_df = filter_redundant_pathways_by_resnik(
    G, 
    relevant_terms, 
    similarity_threshold=0.65# higher = stricter merging
)

In [194]:
significant_enri_path.loc[(significant_enri_path["Description"].isin(filtered_pathways)) & (significant_enri_path["P_value"] <= 0.1)].sort_values(by="Enrichment",ascending=False).drop(columns=["Genes"])#.iloc[-5,-1]

,GeneSet,Description,Enrichment,P_value,FDR,Which_genes
764,Disease,R-HSA-1643685,2690.755444,0.0382,0.434732,"['P35222', 'P04439', 'P01889', 'P07900', 'P163..."
7,ABC-family proteins mediated transport,R-HSA-382556,846.557770,0.0125,0.278500,"['P0CG48', 'P55072', 'P13569', 'Q99942', 'P629..."
183,Assembly of the pre-replicative complex,R-HSA-68867,838.147760,0.0114,0.278500,"['P0CG48', 'P16104', 'P62979', 'P62987', 'P359..."
39,AUF1 (hnRNP D0) binds and destabilizes mRNA,R-HSA-450408,835.334674,0.0095,0.278500,"['P0CG48', 'Q14103', 'P62979', 'P62987', 'P359..."
1047,GSK3B and BTRC:CUL1-mediated-degradation of NF...,R-HSA-9762114,833.877433,0.0092,0.278500,"['P0CG48', 'P62979', 'P62987', 'Q16236', 'P359..."
24,APC/C:Cdc20 mediated degradation of Securin,R-HSA-174154,833.832099,0.0090,0.278500,"['P0CG48', 'P62979', 'P62987', 'P35998', 'Q132..."
2054,Regulation of activated PAK-2p34 by proteasome...,R-HSA-211733,833.832099,0.0090,0.278500,"['P0CG48', 'P62979', 'P62987', 'P35998', 'Q132..."
731,Degradation of CRY and PER proteins,R-HSA-9932298,833.832099,0.0090,0.278500,"['P0CG48', 'P62979', 'P62987', 'P35998', 'Q132..."
88,Activation of NF-kappaB in B cells,R-HSA-1169091,833.663413,0.0099,0.278500,"['P0CG48', 'O14920', 'Q9Y6K9', 'P19838', 'Q042..."
514,Defective CFTR causes cystic fibrosis,R-HSA-5678895,831.905284,0.0092,0.278500,"['P0CG48', 'P55072', 'P13569', 'Q99942', 'P629..."


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.lines as mlines

# Prepare top enriched and significant pathways
top = significant_enri_path.copy()
top = top[(significant_enri_path["Description"].isin(filtered_pathways)) & (top["P_value"] < 0.05)]
top = top.sort_values("P_value").head(20)


# Set up the plot
fig, ax = plt.subplots(figsize=(14, 6))

# Normalize the color scale
norm = mcolors.Normalize(vmin=top["P_value"].min(), vmax=top["P_value"].max())
cmap = cm.get_cmap("RdBu")

# Map the color based on significance
colors = [cmap(norm(val)) for val in top["P_value"]]

# Create the scatter plot
sizes = top["Which_genes"].apply(len)
scatter = ax.scatter(
    x=top["Enrichment"],
    y=top["GeneSet"],
    s=sizes,
    c=colors,
    edgecolor="black"
)

# Invert y-axis
ax.invert_yaxis()

# Add colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("P_value")

# --------------------------
# Add size legend
# --------------------------
for size in [min(sizes), np.median(sizes), max(sizes)]:  # pick example sizes
    ax.scatter([], [], s=size, c="gray", edgecolor="black", label=f"{size}")

legend1 = ax.legend(scatterpoints=1, frameon=True, labelspacing=1, title="Gene count")
ax.add_artist(legend1)

# Labels and layout
ax.set_title("Pathway Enrichment Dot Plot")
ax.set_xlabel("ddPPR Enrichment Score")
ax.set_ylabel("Pathway")

plt.tight_layout()
plt.show()


C:\Users\jpedr\AppData\Local\Temp\ipykernel_20000\2250975057.py:17: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.



In [ ]:
sns.clustermap(resnik_df, cmap="viridis", annot=False)
plt.show()

C:\Users\jpedr\AppData\Roaming\Python\Python311\site-packages\seaborn\matrix.py:560: UserWarning:

Clustering large matrix with scipy. Installing `fastcluster` may give better performance.

C:\Users\jpedr\AppData\Roaming\Python\Python311\site-packages\seaborn\matrix.py:560: UserWarning:

Clustering large matrix with scipy. Installing `fastcluster` may give better performance.



## Shortest Path causal analysis

In [ ]:
# rank seed nodes by dPPR (relevance given the seed set), centrality (dPPR is not immune to centrality scores, which poses the necessity of normalization)
# and distance from the significant target (seed or non-seed)

significant_seeds = wt_df_ddppr.loc[(wt_df_ddppr["Is seed"] == True) & (wt_df_ddppr["P_value_2t"] <= 0.05) & (wt_df_ddppr["ddPPR"] > 0)]
sig_seeds_index = significant_seeds.index.to_list()
print(significant_seeds)

def get_path_length(graph:ig.Graph,ddppr_df:pd.DataFrame,signal_origin=[],seed_list=[],target_list=[]):
    origin_to_seed = np.array(graph.distances(signal_origin, seed_list))
    seed_to_target = np.array(graph.distances(seed_list, target_list))
    combined = []
    for i in range(origin_to_seed.shape[0]):  # For each signal origin
        row = origin_to_seed[i, :].reshape(-1, 1) * seed_to_target  # outer product of seed distances
        combined.append(row)
    return np.array(combined)

from collections import Counter

def get_frequency(graph: ig.Graph, ddppr_df: pd.DataFrame, signal_origin=[], seed_list=[], target_list=[]):
    source_node = signal_origin[0]
    target_node = target_list[0]

    source_name = graph.vs[source_node]["gene_name"]
    target_name = graph.vs[target_node]["gene_name"]
    print(f"Source node is {source_name} and target node is {target_name}")

    seed_shortest_paths = {}
    all_intermediate_nodes = []
    for seed_index in seed_list:
        if seed_index == target_node or seed_index == source_node:
            continue

        source_to_seed_paths = graph.get_all_shortest_paths(v=source_node, to=seed_index)
        seed_to_target_paths = graph.get_all_shortest_paths(v=seed_index, to=target_node)

        full_paths = []
        for s2s in source_to_seed_paths:

            for s2t in seed_to_target_paths:
                # Make sure we don't duplicate the seed node
                if s2s[-1] == s2t[0]:
                    full_path = s2s[:-1] + s2t
                else:
                    full_path = s2s + s2t
                full_paths.append(full_path)
            all_intermediate_nodes.extend(s2s[1:-1])
        
        seed_name = graph.vs[seed_index]["gene_name"] # retrieve the respective seed name
        seed_shortest_paths[seed_name] = full_paths # add shortest paths per seed

        # Extract intermediate nodes (excluding source and target)        
        for s2t in seed_to_target_paths:
            all_intermediate_nodes.extend(s2t[1:-1])
    all_intermediate_nodes = [node for node in all_intermediate_nodes if node not in seed_list] # remove seed nodes from the bridge nodes
    # Count frequency of each intermediate node
    freq_counter = Counter(all_intermediate_nodes)

    return seed_shortest_paths, all_intermediate_nodes

path_nodes = get_frequency(
    graph=cf_connected_graph,
    ddppr_df=wt_df_ddppr,
    signal_origin=[cftr_index],
    seed_list=sig_seeds_index,
    target_list = [cf_connected_graph.vs.find(gene_name="CAPN1").index]
)

# plot the frequency of each bridge node
all_bridge_nodes = pd.Series(path_nodes[1]).value_counts()
all_bridge_names = which_index(all_bridge_nodes.index.to_list(),cf_connected_graph,"gene_name")

node_names = all_bridge_names[:11]
node_freq = all_bridge_nodes[:11]
f,ax = plt.subplots(1,1,figsize=(7,5))

ax.bar(x=node_names,height=node_freq)
#plt.show()

def find_bottleneck(graph:ig.Graph,subgraph_list:list,source_index,target_index,damping=0.85):

    s2t = graph.induced_subgraph(subgraph_list)
    s2t_betweenness = (np.array(s2t.betweenness()) + 1)
    source_index = s2t.vs.find(gene_name=source_index).index
    target_index = s2t.vs.find(gene_name=target_index).index
    if s2t.is_connected():
        print(f"The graph is connected!")
        
        restart = np.full(len(subgraph_list),1e-6)
        restart[[source_index,target_index]] = 5
        ppr = np.array(s2t.personalized_pagerank(reset=restart, damping=damping, directed=False))

        results = pd.DataFrame(data={
            "Gene Name":graph.vs[subgraph_list]["gene_name"],
            "Score":ppr/s2t_betweenness
        }
        )
        return results
    else:
        print("The graph is not connected")
        return []

source_index = cf_connected_graph.vs.find(gene_name="CFTR").index
target_index = cf_connected_graph.vs.find(gene_name="CAPN1").index
subgraph_list = all_bridge_nodes.index.to_list() + sig_seeds_index + [cftr_index]
find_bottleneck(cf_connected_graph,subgraph_list,"CFTR","CAPN1").sort_values(by="Score",ascending=False)

Empty DataFrame
Columns: [UniprotID, Control_PPR, Test_PPR, dPPR_C, dPPR_t, ddPPR, P_value_2t, FDR, Is seed, Gene Name]
Index: []
Source node is CFTR and target node is CAPN1


ValueError: no such vertex

In [ ]:
cf_connected_graph.vs.find(name=nherf1)

ValueError: no such vertex: 'O14745'

ValueError: All arrays must be of the same length

In [ ]:
from kneed import KneeLocator

# Suppose your ddPPR values are in a pandas Series
ddppr_series = np.abs(wt_df_ddppr.loc[is_seed.str.contains("Test",case=True)]["ddPPR"])
abs_ddppr = ddppr_series.abs().sort_values(ascending=False)

# Rank indices (for x-axis)
x = np.arange(1, len(abs_ddppr) + 1)
y = abs_ddppr.values

knee = KneeLocator(x, y, curve="convex", direction="decreasing")
knee_point = knee.knee

knee_index = abs_ddppr.iloc[:knee_point+1].index[-1]

plt.figure(figsize=(10, 6))
plt.plot(x, y, label="|ddPPR| (sorted)", color="blue")
plt.axvline(x=knee_point, color="red", linestyle="--", label=f"Knee point: {knee_point}")
plt.scatter(knee_point, y[knee_point - 1], color="red", s=100, zorder=5)

plt.xlabel("Ranked node index")
plt.ylabel("|ddPPR|")
plt.title("Knee point detection for seed filtering")
plt.legend()
plt.tight_layout()
plt.show()

# Personalized Page Rank for protein relevance identification

## Other codes

### Restart vector permutation

In [66]:
from scipy.stats import norm

restart_vector = np.full(cf_connected_graph.vcount(), 1e-5) # restart vector for the random walk

restart_vector[cftr_index] = 0.07

for interactor in control_seeds["seed"]:
    restart_vector[interactor] = 0.07


"""for i in unique_sec_int:
    restart_vector[i] = 0.001"""

print(f"The non-normalized restart vector entries sum to {restart_vector.sum()}")

# Normalize to sum to 1
restart_vector /= restart_vector.sum()

print(f"The normalized restart vector entries sum to {restart_vector.sum()}")

pagerank_scores = cf_connected_graph.personalized_pagerank(reset=restart_vector,damping=0.85)

iteration = 1000
 # random vector for the random walk

rscores = np.zeros((cf_connected_graph.vcount(),iteration))

# Edge swap 
for i in range(iteration):
    rvector = np.random.permutation(restart_vector)
    rscore = cf_connected_graph.personalized_pagerank(reset=rvector, damping=0.7,directed=False)
    rscores[:, i] = rscore  # Save raw scores for each node

# Compute p-values for each node: how often the randomized score is >= real one
p_values = np.mean(rscores >= np.array(pagerank_scores)[:, None], axis=1)

nodes_ids = cf_connected_graph.vs["name"] # get the node IDs from the graph

# Calculate the z score for each node
z_score = (pagerank_scores - np.mean(rscores, axis=1))/np.std(rscores,axis=1)
p_values_one_tailed = 1 - norm.cdf(z_score)
np.sum(p_values_one_tailed <= 0.05)

final_score = pd.DataFrame(data=list(zip(nodes_ids,pagerank_scores,p_values_one_tailed)),columns=["UniprotID","PageRank","P_value"]) # create a df with the scores and p-values

print(f"Nodes for which p <= 0.05: *{np.sum(np.sum(rscores >= np.array(pagerank_scores)[:, None],axis=1)/iteration <= 0.05)}* nodes")


The non-normalized restart vector entries sum to 3.0898900000000005
The normalized restart vector entries sum to 0.9999999999999999
Nodes for which p <= 0.05: *51* nodes


### Optimization: add centrality and bias context to the restart probabilities

In [ ]:
cf_nodes = cf_connected_graph.vcount()

cf_neighbors = len(cf_connected_graph.neighbors(cftr_index, mode="all"))

cf_membrane_neighbors = len(node_intersect_wt)

print(f"Out of {cf_nodes} nodes in the connected graph, {cf_neighbors} are CFTR neighbors and {cf_membrane_neighbors} are membrane interactors")

cf_non_seeds = cf_nodes - cf_neighbors

cf_nonmemb_seeds = cf_neighbors - cf_membrane_neighbors

print(f"There are {cf_membrane_neighbors} seeds and {cf_nonmemb_seeds} non-membrane seeds")

seed_weight = 0.5

memb_weight = seed_weight * 4/5
nonmemb_seeds_weight = seed_weight * 1/5
total_nonseed_weight = cf_non_seeds * 1e-5
print(f"Membrane seed weight per node is {memb_weight/cf_membrane_neighbors:.4f} and non-membrane seed weight is {nonmemb_seeds_weight/cf_nonmemb_seeds:.4f}")

In [ ]:
# ----- Setup and Parameters -----
alpha = 0.85
seed_mass = 0.5
epsilon = 1e-5


# ----- Inverse Centrality -----
"""betweenness = cf_centralities_measure["Betweeness"] # check for large differences 
inv_centrality = 1 / (betweenness)
inv_centrality.name = "Inverse Centrality"
print(f"Total centrality score sum for seed nodes: {inv_centrality[seeds].sum():.4f}")"""

# ----- Restart weights for bias estimation -----
membrane_mass = seed_mass * 4/5
non_membrane_mass = seed_mass * 1/5

memb_seed_weight = membrane_mass / len(node_intersect_wt + [cftr])
non_memb_seed_weight = non_membrane_mass / len(wt_nonmemb_indices)

print(f"Seed weights for bias run: {memb_seed_weight:.4f} (membrane), {non_memb_seed_weight:.4f} (non-membrane)")

# ----- Control (unbiased) PPR -----
ctrl_restart_vector = np.full(cf_connected_graph.vcount(), 1e-5)
ctrl_restart_vector /= ctrl_restart_vector.sum()

ctrl_pagerank_scores = cf_connected_graph.personalized_pagerank(
    reset=ctrl_restart_vector, damping=alpha, directed=False
)

# ----- Biased PPR (used to measure overbias) -----
biased_restart_vector = np.full(cf_connected_graph.vcount(), 1e-5)

for index in membint_index + [cftr_index]:
    biased_restart_vector[index] = memb_seed_weight

for index in wt_nonmemb_indices:
    biased_restart_vector[index] = non_memb_seed_weight

biased_restart_vector /= biased_restart_vector.sum()

es_pagerank_scores = cf_connected_graph.personalized_pagerank(
    reset=biased_restart_vector, damping=alpha, directed=False
)

# ----- Compute Bias and Corrected Weights -----
bias = np.array(es_pagerank_scores) - np.array(ctrl_pagerank_scores)
inv_bias = 1 / (bias + epsilon)

# Final corrected restart vector: inverse centrality × inverse bias
corrected_restart_vector = np.full(cf_connected_graph.vcount(), 1e-5)

for idx in seeds:
    corrected_restart_vector[idx] = inv_centrality[idx] * inv_bias[idx]

corrected_restart_vector /= corrected_restart_vector.sum()

# ----- Final PPR with corrected restart vector -----
clean_pagerank_scores = cf_connected_graph.personalized_pagerank(
    reset=corrected_restart_vector, damping=alpha, directed=False
)

# ----- Plot: corrected PPR gain vs centrality -----
plt.figure(figsize=(10, 6))
plt.scatter(
    x=np.log(cf_centralities_measure["Betweeness"] + epsilon),
    y=np.array(clean_pagerank_scores) - np.array(ctrl_pagerank_scores),
    alpha=0.6
)
plt.xlabel("log(Betweenness Centrality)")
plt.ylabel("PPR Shift (Corrected - Control)")
plt.title("Overbias-corrected PPR Gain vs Centrality")
plt.grid(True)
plt.tight_layout()
plt.show()


## Edge swap for unaltered node connectivity

In [67]:
gamma = 1
betweenness = cf_centralities_measure.loc[cf_centralities_measure.index.isin(control_seeds["seed"]+[cftr_index])]["Betweeness"]**gamma
inv_centrality = 1 / betweenness
inv_centrality *= 1/inv_centrality.sum() 
seed_index = inv_centrality.index.to_list()
#print(inv_centrality)
inv_centrality.name = "Inverse Centrality"

#print(inv_centrality.sum())
#print(0.07*len(membint_index+[cftr_index]),0.01*len(wt_nonmemb_indices),0.07*len(membint_index+[cftr_index])+0.01*len(wt_nonmemb_indices))

In [69]:
from statsmodels.stats.multitest import multipletests

def PPR_difusion(g:ig.Graph,seed_set1,seed_set2,background,centrality,directed,iterations:int,alpha:float,gamma:float,attribute_display:str):
    
    seeds = seed_set1 + seed_set2 # Combine the two seed sets

    node_ids = g.vs[attribute_display] # get the node IDs from the graph

    # Corretion of seed centralities
    inv_centrality = 1 / centrality**gamma 
    inv_centrality.name = "Inverse Centrality"

    # Initiate the restart vector
    restart_vector = np.full(
        g.vcount(),
        background
    )

    # Load the seed restart weights
    if seed_set1:
        restart_vector[seed_set1] = 0.07 # attribute membrane seeds full restart p
    if seed_set2:   
        restart_vector[seed_set2] = 0.01 # attribute non-membrane seeds lower restart p 
        
    print(f"The WT non-normalized restart vector entries sum to {restart_vector.sum()}")

    restart_vector /= restart_vector.sum() # Normalize to sum to 1

    # Run the actual PPR
    es_pagerank_scores = np.array( # convert to np array for flexibility
        g.personalized_pagerank( # personalized page rank
        reset=restart_vector,
        damping=alpha,
        directed=directed))
    
    # Run a control PPR with unbiased restart probabilities
    ctrl_restart_vector = np.full(g.vcount(),
                              1e-12) # restart vector for the random walk

    ctrl_pagerank_scores = np.array(g.personalized_pagerank(reset=ctrl_restart_vector, damping=alpha, directed=directed))

    dppr = (es_pagerank_scores - ctrl_pagerank_scores) / ctrl_pagerank_scores # calculate the difference between the two pagerank scores

    # Run multiple network rewirings to calculate significance scores
    null_scores = np.zeros((g.vcount(), iterations)) # Matrix initialization to collect null scores from the rewired graph
    null_dppr_scores = np.zeros((g.vcount(), iterations)) # Matrix initialization to collect null difference scores from the rewired graph
    for i in tqdm(range(iterations)):
        # Rewire the graph while preserving node degrees
        rewired_graph = g.copy()
        rewired_graph.rewire(mode="simple", n=10 * rewired_graph.ecount())
        
        # Run PPR on the rewired graph using the same restart vector
        null_ppr = np.array(rewired_graph.personalized_pagerank(reset=restart_vector, damping=alpha, directed=False))
        null_ctrl_ppr = np.array(rewired_graph.personalized_pagerank(reset=ctrl_restart_vector, damping=alpha, directed=False))
        null_scores[:, i] = null_ppr
        null_dppr_scores[:, i] = (null_ppr - null_ctrl_ppr) / null_ctrl_ppr

    # Compute z-scores and p-values
    z_scores = (dppr  - np.mean(null_dppr_scores, axis=1)) / np.std(null_dppr_scores, axis=1)
    p_values_one_tailed = (1 - norm.cdf(z_scores))
    _, fdr_corrected_pvals, _, _ = multipletests(p_values_one_tailed, method='fdr_bh') # Multiple testing correction 

    final_score = pd.DataFrame(data=list(zip(node_ids,es_pagerank_scores,dppr,p_values_one_tailed,fdr_corrected_pvals)),columns=["UniprotID","PageRank","dPPR","P_value","FDR"]) # create a df with the scores and p-values
    final_score["Is seed"] = final_score.index.isin(seeds) # add a column to indicate if the node is a seed or not
    return final_score


In [71]:

final_score = PPR_difusion(g=cf_connected_graph,
                    seed_set1=control_seeds["seed"]+[cftr_index],
                    seed_set2=[],
                    background=1e-6,
                    centrality=cf_centralities_measure["Betweeness"],
                    directed=False,
                    iterations=1000,
                    alpha=0.4,
                    gamma=1,
                    attribute_display="name")



The WT non-normalized restart vector entries sum to 3.0809889999999998


  0%|          | 2/1000 [00:00<01:16, 13.12it/s]

100%|██████████| 1000/1000 [01:13<00:00, 13.53it/s]


In [98]:
final_score.sort_values(by="dPPR", ascending=False)

,UniprotID,PageRank,dPPR,P_value,FDR,Is seed
1004,Q9UBM7,1.373665e-02,21.547283,0.437734,0.819371,True
1030,Q9Y2T7,1.371462e-02,20.903167,0.137556,0.819371,True
1018,Q9NPF4,1.402845e-02,20.764193,0.161926,0.819371,True
930,P35241,1.412053e-02,20.761923,0.157055,0.819371,True
914,P30530,1.374170e-02,20.089286,0.149175,0.819371,True
...,...,...,...,...,...,...
796,P98088,1.013391e-06,-0.998464,0.573398,0.819371,False
710,Q9UEW3,1.337243e-06,-0.998671,0.629983,0.819371,False
760,Q8IWL2,8.961888e-07,-0.998710,0.608477,0.819371,False
681,P08185,7.516743e-07,-0.999023,0.623939,0.819371,False


In [143]:
final_score_df = PPR_difusion(g=cf_connected_graph,
                    seed_set1=test_seeds["seed"]+[cftr_index],
                    seed_set2=[],
                    background=1e-6,
                    centrality=cf_centralities_measure["Betweeness"],
                    directed=False,
                    iterations=1000,
                    alpha=0.4,
                    gamma=1,
                    attribute_display="name")

The WT non-normalized restart vector entries sum to 13.300843000000002


100%|██████████| 1000/1000 [00:34<00:00, 29.26it/s]


In [144]:
print(f"Page Rank scores for membrane-CFTR-biased algorithm:\nSignificant nodes at p ≤ 0.05: {np.sum(final_score.P_value <= 0.05)}\nSignificant nodes at FDR ≤ 0.05: {np.sum(final_score.FDR <= 0.05)}")
#print(f"Page Rank scores for membrane-CFTR-biased algorithm:\nSignificant nodes at p ≤ 0.05: {np.sum(final_score_df.P_value <= 0.05)}\nSignificant nodes at FDR ≤ 0.05: {np.sum(final_score_df.FDR <= 0.05)}")

Page Rank scores for membrane-CFTR-biased algorithm:
Significant nodes at p ≤ 0.05: 53
Significant nodes at FDR ≤ 0.05: 15


In [145]:
relevant_proteins = final_score.loc[(final_score["FDR"] <= 0.05) & (final_score["dPPR"] > 0) & (final_score["Is seed"] == False)]["UniprotID"].to_list()
for protein in relevant_proteins:
    print(protein)

P07900
Q6NUS6
P38646
P27797
Q9Y3M8
P40692
P07711
P51957
Q15077
Q15393
P62854
P04632
P61421


In [142]:
f, ax = plt.subplots(1,1,figsize=(7,5))
results = final_score.copy()
results.sort_values(by="dPPR", ascending=False, inplace=True)
results.index = range(len(results))
ax.plot(results.index.to_list(),results["dPPR"],marker="o",alpha = 0.3)
ax.scatter(results.loc[results["P_value"]<=0.05].index.to_list(),results.loc[results["P_value"]<=0.05]["dPPR"],c="yellow",alpha=0.8)
plt.show()

### PPR statistics summary

#### WT run summary

In [94]:
fig = go.Figure(data=go.Scatter(x=final_score.loc[final_score["Is seed"]==False]["P_value"],
                                y=final_score.loc[final_score["Is seed"]==False]["dPPR"],
                                mode='markers',
                                hovertext=final_score.loc[final_score["Is seed"]==False]["UniprotID"],
                                name="All nodes"))

fig.update_layout(title='PPR score vs P value',
    yaxis_title='dPPR Score (relevance)',
    xaxis_title='P value',
    width=800,  # Width in pixels
    height=600  # Height in pixels
)

fig.add_trace(go.Scatter(
    x=final_score.loc[(final_score["Is seed"]==True) & (~final_score["UniprotID"].isin([cftr]))]["P_value"],
    y=final_score.loc[(final_score["Is seed"]==True) & (~final_score["UniprotID"].isin([cftr]))]["dPPR"],
    mode='markers',
    name='CFTR Neighbors',
    marker=dict(color='green', size=7, symbol='square'),
    hovertext=final_score.loc[(final_score["Is seed"]==True) & (~final_score["UniprotID"].isin([cftr]))]["UniprotID"]
))

fig.add_trace(go.Scatter(
    x=final_score.loc[final_score["UniprotID"] == cftr]["P_value"],
    y=final_score.loc[final_score["UniprotID"] == cftr]["dPPR"],
    mode='markers',
    name='CFTR',
    marker=dict(color='red', size=10, symbol='star'),
    hovertext=cf_centralities_measure.loc[cf_centralities_measure["UniprotID"] == cftr]["UniprotID"].values[0]
))

fig.add_shape(
    type='line',
    x0=0.05, x1=0.05,
    y0=0, y1=1,  # In yref="paper", 0–1 spans the full y-axis height
    line=dict(color='Red', dash='dash'),
    xref='x',
    yref='paper'
)

fig.add_shape(
    type='line',
    x0=0.0, x1=1,
    y0=0, y1=0,  # In yref="paper", 0–1 spans the full y-axis height
    line=dict(color='Black', dash='dash'),
    xref='paper',
    yref='y'
)

fig.show()


##### Centrality distribution of PPR scores

In [93]:
score = "dPPR"

fig = go.Figure(data=go.Scatter(y=final_score.loc[(cf_centralities_measure["Betweeness"]>0) & (final_score["Is seed"] == False)][score],
                                x=np.log10(cf_connected_graph.betweenness())[(np.log10(cf_connected_graph.betweenness()) > 0) & (final_score["Is seed"] == False)],
                                mode='markers',
                                hovertext=final_score.loc[(cf_centralities_measure["Betweeness"]>0) & (final_score["Is seed"] == False) ]["UniprotID"],
                                name="All nodes"))

# Update figure's layout
fig.update_layout(title='PPR score vs Betweeness',
    yaxis_title=f'{score} Score (relevance)',
    xaxis_title='Betweenes (log10 scale)',
    width=800,  # Width in pixels
    height=600  # Height in pixels
)

# Add a trace to identify significant (p_val <= 0.05) nodes
fig.add_trace(go.Scatter(
    y=final_score.loc[(final_score["P_value"] <= 0.05)][score],
    x=np.log10(cf_connected_graph.betweenness())[(final_score["P_value"] <= 0.05)],
    mode='markers',
    name='Significant seed nodes',
    marker=dict(color='cadetblue', size=7, symbol='circle'),
    hovertext=[f"{id} - {p_val}" for id,p_val in zip(final_score.loc[(final_score["P_value"] <= 0.05)]["UniprotID"],final_score.loc[(final_score["P_value"] <= 0.05) & (final_score["Is seed"] == True)]["P_value"])]
))

# Add a trace to identify non-significant seed nodes
"""fig.add_trace(go.Scatter(
    y=final_score.loc[(final_score["P_value"] > 0.05) & (final_score["Is seed"] == True)][score],
    x=np.log10(cf_centralities_measure.loc[(final_score["P_value"] > 0.05)]["Betweeness"]),
    mode='markers',
    name='Non-significant nodes',
    marker=dict(color='wheat', size=7, symbol='circle'),
    hovertext=[f"{id} - {p_val}" for id,p_val in zip(final_score.loc[(final_score["P_value"] > 0.05) & (final_score["Is seed"] == True)]["UniprotID"],final_score.loc[(final_score["P_value"] <= 0.05)& (final_score["Is seed"] == False)]["P_value"])]
))"""

# Add a trace to identify FDR corrected significant nodes
"""fig.add_trace(go.Scatter(
    y=final_score.loc[final_score["FDR score"] <= 0.05][score],
    x=np.log10(cf_centralities_measure.loc[final_score["FDR score"] <= 0.05]["Betweeness"]),
    mode='markers',
    name='FDR significant nodes',
    marker=dict(color='forestgreen', size=7, symbol='circle'),
    hovertext=[f"{id} - {p_val}" for id,p_val in zip(final_score.loc[final_score["FDR score"] <= 0.05]["UniprotID"],final_score.loc[final_score["FDR score"] <= 0.05]["FDR score"])]
))
"""

# Add a trace to identify non-seed significant nodes
"""fig.add_trace(go.Scatter(
    y=final_score.loc[(final_score["P_value"] <= 0.05) & (final_score["Is seed"] == False)][score],
    x=np.log10(cf_centralities_measure.loc[(final_score["P_value"] <= 0.05) & (final_score["Is seed"] == False)]["Betweeness"]),
    mode='markers',
    name='Non-seed significant nodes',
    marker=dict(color='purple', size=7, symbol='circle'),
    hovertext=[f"{id} - {p_val}" for id,p_val in zip(non_seed_rel,final_score.loc[(final_score["P_value"] <= 0.05) & (final_score["Is seed"] == False)]["P_value"])]
))"""

"""fig = go.Figure(data=go.Scatter(y=final_score.loc[~final_score["UniprotID"].isin([name for name in memint_name]+neighbor_names+[cftr])]["PageRank"],
                                x=np.log10(cf_centralities_measure.loc[~cf_centralities_measure["UniprotID"].isin([name for name in memint_name]+neighbor_names+[cftr])]["Betweeness"]),
                                mode='markers',
                                hovertext=cf_connected_graph.vs["name"],
                                name="All nodes"))
fig.update_layout(title='PPR score vs Betweeness',
    yaxis_title='Page Rank Score (relevance)',
    xaxis_title='Betweenes (log10 scale)',
    width=800,  # Width in pixels
    height=600  # Height in pixels
)

fig.add_trace(go.Scatter(
    y=final_score.loc[final_score["UniprotID"].isin(cf_neighbors)]["PageRank"],
    x=np.log10(cf_centralities_measure.loc[cf_centralities_measure["UniprotID"].isin(cf_neighbors)]["Betweeness"]),#final_score.loc[final_score["UniprotID"].isin(cf_neighbors)]["PageRank"],
    mode='markers',
    name='Membrane interactors',
    marker=dict(color='purple', size=7, symbol='square'),
    hovertext=cf_neighbors
))
"""
fig.show()

print(ezr,nherf1,cftr)

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\4002027991.py:4: RuntimeWarning:

divide by zero encountered in log10

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\4002027991.py:20: RuntimeWarning:

divide by zero encountered in log10



P15311 O14745 P13569


In [91]:
print(f"Node dPPR distribution over log10(Betweeness) Centrality")

final_score_noseeds = final_score#.loc[final_score["Is seed"] == False]

ppr0_1 = final_score_noseeds.loc[(np.log10(cf_connected_graph.betweenness())>=0) & (np.log10(cf_connected_graph.betweenness())<1)]

ppr1_2 = final_score_noseeds.loc[(np.log10(cf_connected_graph.betweenness())>=1) & (np.log10(cf_connected_graph.betweenness())<2)]

ppr2_3 = final_score_noseeds.loc[(np.log10(cf_connected_graph.betweenness())>=2) & (np.log10(cf_connected_graph.betweenness())<3)]

ppr3_4 = final_score_noseeds.loc[(np.log10(cf_connected_graph.betweenness())>=3) & (np.log10(cf_connected_graph.betweenness())<4)]

f, ax = plt.subplots(1,1,figsize=(7,5))

ax.boxplot([ppr0_1["dPPR"],ppr1_2["dPPR"],ppr2_3["dPPR"],ppr3_4["dPPR"]],labels=["0-1","1-2","2-3","3->4"])
ax.set_title("dPPR score distribution by log10(Betweeness) Centrality")

plt.show()

Node dPPR distribution over log10(Betweeness) Centrality


C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\769699790.py:5: RuntimeWarning:

divide by zero encountered in log10

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\769699790.py:7: RuntimeWarning:

divide by zero encountered in log10

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\769699790.py:9: RuntimeWarning:

divide by zero encountered in log10

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\769699790.py:11: RuntimeWarning:

divide by zero encountered in log10

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\769699790.py:15: MatplotlibDeprecationWarning:

The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.



In [83]:
import plotly.express as px

final_score_seeds = final_score.loc[final_score["Is seed"] == True]

# Create violin plot
fig = px.violin(final_score,
                y="dPPR",
                x="Is seed", 
                box=True, 
                title="Seed vs Non-seed dPPR",
                width = 600,
                height = 600
)

fig.show()

#### F508del run summary

In [146]:
fig = go.Figure(data=go.Scatter(x=final_score_df.loc[final_score_df["Is seed"]==False]["P_value"],
                                y=final_score_df.loc[final_score_df["Is seed"]==False]["dPPR"],
                                mode='markers',
                                hovertext=final_score_df.loc[final_score_df["Is seed"]==False]["UniprotID"],
                                name="All nodes"))

fig.update_layout(title='PPR score vs P value',
    yaxis_title='dPPR Score (relevance)',
    xaxis_title='P value',
    width=800,  # Width in pixels
    height=600  # Height in pixels
)

fig.add_trace(go.Scatter(
    x=final_score_df.loc[(final_score_df["Is seed"]==True) & (~final_score["UniprotID"].isin([cftr]))]["P_value"],
    y=final_score_df.loc[(final_score_df["Is seed"]==True) & (~final_score["UniprotID"].isin([cftr]))]["dPPR"],
    mode='markers',
    name='CFTR Neighbors',
    marker=dict(color='green', size=7, symbol='square'),
    hovertext=final_score_df.loc[(final_score_df["Is seed"]==True) & (~final_score_df["UniprotID"].isin([cftr]))]["UniprotID"]
))

fig.add_trace(go.Scatter(
    x=final_score_df.loc[final_score_df["UniprotID"] == cftr]["P_value"],
    y=final_score_df.loc[final_score_df["UniprotID"] == cftr]["dPPR"],
    mode='markers',
    name='CFTR',
    marker=dict(color='red', size=10, symbol='star'),
    hovertext=cf_centralities_measure.loc[cf_centralities_measure["UniprotID"] == cftr]["UniprotID"].values[0]
))

fig.add_shape(
    type='line',
    x0=0.05, x1=0.05,
    y0=0, y1=1,  # In yref="paper", 0–1 spans the full y-axis height
    line=dict(color='Red', dash='dash'),
    xref='x',
    yref='paper'
)

fig.add_shape(
    type='line',
    x0=0.0, x1=1,
    y0=0, y1=0,  # In yref="paper", 0–1 spans the full y-axis height
    line=dict(color='Black', dash='dash'),
    xref='paper',
    yref='y'
)

fig.show()


##### Centrality distribution of PPR scores

In [148]:
score = "dPPR"

fig = go.Figure(data=go.Scatter(y=final_score_df.loc[(cf_centralities_measure["Betweeness"]>0) & (final_score_df["Is seed"] == False)][score],
                                x=np.log10(cf_connected_graph.betweenness())[(np.log10(cf_connected_graph.betweenness()) > 0) & (final_score_df["Is seed"] == False)],
                                mode='markers',
                                hovertext=final_score.loc[(cf_centralities_measure["Betweeness"]>0) & (final_score_df["Is seed"] == False) ]["UniprotID"],
                                name="All nodes"))

# Update figure's layout
fig.update_layout(title='PPR score vs Betweeness',
    yaxis_title=f'{score} Score (relevance)',
    xaxis_title='Betweenes (log10 scale)',
    width=800,  # Width in pixels
    height=600  # Height in pixels
)

# Add a trace to identify significant (p_val <= 0.05) nodes
fig.add_trace(go.Scatter(
    y=final_score_df.loc[(final_score_df["P_value"] <= 0.05)][score],
    x=np.log10(cf_connected_graph.betweenness())[(final_score["P_value"] <= 0.05)],
    mode='markers',
    name='Significant seed nodes',
    marker=dict(color='cadetblue', size=7, symbol='circle'),
    hovertext=[f"{id} - {p_val}" for id,p_val in zip(final_score_df.loc[(final_score_df["P_value"] <= 0.05)]["UniprotID"],final_score_df.loc[(final_score_df["P_value"] <= 0.05) & (final_score_df["Is seed"] == True)]["P_value"])]
))

# Add a trace to identify FDR corrected significant nodes
"""fig.add_trace(go.Scatter(
    y=final_score.loc[final_score["FDR score"] <= 0.05][score],
    x=np.log10(cf_centralities_measure.loc[final_score["FDR score"] <= 0.05]["Betweeness"]),
    mode='markers',
    name='FDR significant nodes',
    marker=dict(color='forestgreen', size=7, symbol='circle'),
    hovertext=[f"{id} - {p_val}" for id,p_val in zip(final_score.loc[final_score["FDR score"] <= 0.05]["UniprotID"],final_score.loc[final_score["FDR score"] <= 0.05]["FDR score"])]
))
"""

# Add a trace to identify non-seed significant nodes
"""fig.add_trace(go.Scatter(
    y=final_score.loc[(final_score["P_value"] <= 0.05) & (final_score["Is seed"] == False)][score],
    x=np.log10(cf_centralities_measure.loc[(final_score["P_value"] <= 0.05) & (final_score["Is seed"] == False)]["Betweeness"]),
    mode='markers',
    name='Non-seed significant nodes',
    marker=dict(color='purple', size=7, symbol='circle'),
    hovertext=[f"{id} - {p_val}" for id,p_val in zip(non_seed_rel,final_score.loc[(final_score["P_value"] <= 0.05) & (final_score["Is seed"] == False)]["P_value"])]
))"""

"""fig = go.Figure(data=go.Scatter(y=final_score.loc[~final_score["UniprotID"].isin([name for name in memint_name]+neighbor_names+[cftr])]["PageRank"],
                                x=np.log10(cf_centralities_measure.loc[~cf_centralities_measure["UniprotID"].isin([name for name in memint_name]+neighbor_names+[cftr])]["Betweeness"]),
                                mode='markers',
                                hovertext=cf_connected_graph.vs["name"],
                                name="All nodes"))
fig.update_layout(title='PPR score vs Betweeness',
    yaxis_title='Page Rank Score (relevance)',
    xaxis_title='Betweenes (log10 scale)',
    width=800,  # Width in pixels
    height=600  # Height in pixels
)

fig.add_trace(go.Scatter(
    y=final_score.loc[final_score["UniprotID"].isin(cf_neighbors)]["PageRank"],
    x=np.log10(cf_centralities_measure.loc[cf_centralities_measure["UniprotID"].isin(cf_neighbors)]["Betweeness"]),#final_score.loc[final_score["UniprotID"].isin(cf_neighbors)]["PageRank"],
    mode='markers',
    name='Membrane interactors',
    marker=dict(color='purple', size=7, symbol='square'),
    hovertext=cf_neighbors
))
"""
fig.show()

print(ezr,nherf1,cftr)

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\282770571.py:4: RuntimeWarning:

divide by zero encountered in log10

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\282770571.py:20: RuntimeWarning:

divide by zero encountered in log10



P15311 O14745 P13569


In [153]:
print(f"Node dPPR distribution over log10(Betweeness) Centrality")

final_score_noseeds_df = final_score_df#.loc[final_score_df["Is seed"] == False]

ppr0_1 = final_score_noseeds.loc[(np.log10(cf_connected_graph.betweenness())>=0) & (np.log10(cf_connected_graph.betweenness())<1)]

ppr1_2 = final_score_noseeds.loc[(np.log10(cf_connected_graph.betweenness())>=1) & (np.log10(cf_connected_graph.betweenness())<2)]

ppr2_3 = final_score_noseeds.loc[(np.log10(cf_connected_graph.betweenness())>=2) & (np.log10(cf_connected_graph.betweenness())<3)]

ppr3_4 = final_score_noseeds.loc[(np.log10(cf_connected_graph.betweenness())>=3) & (np.log10(cf_connected_graph.betweenness())<4)]

f, ax = plt.subplots(1,1,figsize=(7,5))

ax.boxplot([ppr0_1["dPPR"],ppr1_2["dPPR"],ppr2_3["dPPR"],ppr3_4["dPPR"]],labels=["0-1","1-2","2-3","3->4"])
ax.set_title("dPPR score distribution by log10(Betweeness) Centrality")

plt.show()

Node dPPR distribution over log10(Betweeness) Centrality


C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\3492977079.py:5: RuntimeWarning:

divide by zero encountered in log10

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\3492977079.py:7: RuntimeWarning:

divide by zero encountered in log10

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\3492977079.py:9: RuntimeWarning:

divide by zero encountered in log10

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\3492977079.py:11: RuntimeWarning:

divide by zero encountered in log10

C:\Users\jpedr\AppData\Local\Temp\ipykernel_6576\3492977079.py:15: MatplotlibDeprecationWarning:

The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.



## Unbiased background PPR scores

In [159]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import linregress

# --- Personalized PageRank ---
ctrl_restart_vector = np.full(cf_connected_graph.vcount(), 1e-5)  # uniform restart
alpha = 0.45  # damping factor
ctrl_pagerank_scores = cf_connected_graph.personalized_pagerank(
    reset=ctrl_restart_vector,
    damping=alpha,
    directed=False
)

# --- Combine into dataframe ---
nodes_ids = cf_connected_graph.vs["name"]
ctrl_final_scores = pd.DataFrame({
    "UniprotID": nodes_ids,
    "PageRank": ctrl_pagerank_scores,
    "Betweeness": np.array(cf_connected_graph.betweenness())
})

# --- Filter out zero betweenness ---
mask = ctrl_final_scores["Betweeness"] > 0
x = np.array(cf_connected_graph.betweenness())[mask]
y = ctrl_final_scores.loc[mask, "PageRank"].values

# --- Power-law fit in log-log space ---
logx, logy = np.log10(x), np.log10(y)
slope, intercept, r_value, p_value, std_err = linregress(logx, logy)
b = slope
a = 10**intercept  # since log10(y) = intercept + slope*log10(x)

print(f"Fit: PPR ≈ {a:.3e} * Betweenness^{b:.2f}, R² = {r_value**2:.3f}")

# --- Generate fitted curve ---
x_fit = np.linspace(x.min(), x.max(), 10000)
y_fit = a * x_fit**b

# --- Plot ---
fig = go.Figure()

# Scatter plot
fig.add_trace(go.Scatter(
    x=np.log10(x),
    y=y,
    mode='markers',
    hovertext=ctrl_final_scores.loc[mask, "UniprotID"],
    name="Nodes"
))

# Fitted curve
fig.add_trace(go.Scatter(
    x=np.log10(x_fit),
    y=y_fit,
    mode='lines',
    name='Fitted Curve',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title=f"PPR ≈ {a:.2e} * Betweenness<sup>{b:.2f}</sup> (R²={r_value**2:.2f})",
    xaxis_title="Betweenness (log10 scale)",
    yaxis_title="PageRank Score",
    width=800,
    height=600
)

fig.show()


Fit: PPR ≈ 3.773e-04 * Betweenness^0.16, R² = 0.749


### Optimization of the PPR algorithm: exploring the parameter space

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cftr_p = np.linspace(0.01,0.1,10)
restart_vector[cftr_index] = 0.07

# Create 10 damping values

opt_scores = np.zeros((cf_connected_graph.vcount(), len(cftr_p)))

# Calculate PPR for each damping value
for i, probability in enumerate(cftr_p):
    restart_vector[wt_nonmemb_indices] = 0.01
    scores = cf_connected_graph.personalized_pagerank(reset=restart_vector, damping=0.85, directed=False)
    opt_scores[:, i] = scores

# Plot histograms of PPR scores at each damping value
fig, ax = plt.subplots(2, 5, figsize=(14, 6))
ax = ax.flatten()

for i in range(len(cftr_p)):
    ax[i].hist(opt_scores[:, i], bins=30, color='skyblue', edgecolor='black')
    ax[i].set_title(f"prob={cftr_p[i]:.3f}")
    ax[i].set_xlabel("PPR score")
    ax[i].set_ylabel("Frequency")
    #ax[i].set_ylim(0,150)

fig.suptitle("Optimization for CFTR's neighbors initial probabilities")
plt.tight_layout()
plt.show()


In [ ]:
wt_cyfy_element = pd.read_csv("WT_CyFy_Map/CFTR_cp17-elementExport.csv")
wt_cyfy_element.loc[wt_cyfy_element["type"]=="Protein"]

def get_reference(reference,ref_name):
    if pd.isna(reference):
        #print("No reference")
        return None
    split_ref = reference.split(";")
    for element in split_ref:
        if ref_name in element.strip():
            #print(element)
            ref = element.split(":")[1]
            return ref
        else:
            #print("No UNIPROT id")
            return None

wt_cyfy_element["UniprotID"] = wt_cyfy_element["references"].apply(lambda x: get_reference(x,"UNIPROT"))
      


            

In [ ]:
df_cyfy = pd.read_csv("F508del_CyFy_Map/F508del_cp21-networkExport.csv")
df_cyfy_elements = pd.read_csv("F508del_CyFy_Map/F508del_cp21-elementExport.csv")


# Pankow CFTR network analysis

In [ ]:
def instantiate_graph(txt_file):
    # Load the file
    edges = []
    confidences = []

    with open(txt_file, "r") as f:
        for line in f:
            if line.startswith("*edges") or not line.strip():
                continue
            elif line.startswith("*nodes"):
                break
            parts = line.strip().split('\t')
            node1 = parts[0]
            node2 = parts[1]
            confidence = None
            for p in parts:
                if p.startswith("c "):
                    confidence = float(p.split()[1])
            edges.append((node1, node2))
            confidences.append(confidence)

    # Build graph
    g = ig.Graph()
    node_names = list(set([e[0] for e in edges] + [e[1] for e in edges]))
    g.add_vertices(node_names)
    g.add_edges(edges)
    g.es["confidence"] = confidences

    return g

wt_pankow = instantiate_graph("Pankow WT interactome/netQuant/wt_network7.txt")

df_pankow = instantiate_graph("Pankow F508del interactome/netQuant/24h_net_f1.txt")

print(wt_pankow.summary())
print(wt_pankow.vs["name"])
print("\n")
print(df_pankow.summary())
print(len(df_pankow.vs["name"]))


In [ ]:
import requests
import pandas as pd
from io import StringIO

def query_uniprot_batch(gene_symbols):
    results = []
    for i in range(0, len(gene_symbols), 20):  # batch of 20
        batch = gene_symbols[i:i+20]
        query = "(" + " OR ".join([f'gene_exact:{symbol}' for symbol in batch]) + ") AND organism_id:9606"

        url = "https://rest.uniprot.org/uniprotkb/search"
        params = {
            "query": query,
            "format": "tsv",
            "fields": "accession,gene_names"
        }

        response = requests.get(url, params=params)
        if response.ok:
            df = pd.read_csv(StringIO(response.text), sep="\t")
            results.append(df)
        else:
            print(f"Batch failed: {response.status_code}")

    # Combine all batches
    if results:
        df = pd.concat(results, ignore_index=True)
        df_exploded = df.assign(Gene_Symbol=df['Gene Names'].str.split()).explode('Gene_Symbol')
        df_unique = df_exploded.drop_duplicates(subset='Gene_Symbol')
        df_filtered = df_unique[df_unique['Gene_Symbol'].isin(gene_symbols)]
        return df_filtered
    else:
        return pd.DataFrame()


print(f'{len(wt_pankow.vs["name"])} gene symbols to query in UniProt')
wt_results = query_uniprot_batch(wt_pankow.vs["name"])
print(f'{len(df_pankow.vs["name"])} gene symbols to query in UniProt')
df_results = query_uniprot_batch(df_pankow.vs["name"])

In [ ]:
wt_pankow_uniprot = wt_results["Entry"].to_list()

id_in_graph = [id for id in wt_pankow_uniprot if id in cf_connected_graph.vs["name"]]

print(len(id_in_graph))

[node for node in which_attribute(id_in_graph,cf_connected_graph,"name") if node in neighbor_names]

neighbor_names

In [ ]:
df_pankow_uniprot = df_results["Entry"].to_list()

id_in_graph = [id for id in df_pankow_uniprot if id in cf_connected_graph.vs["name"]]

print(len(id_in_graph))

# CF-Connected graph: selection of the most relevant proteins based on how many CF-related neighbors they have

<!-- # Identify possible CF-related proteins through their degree of interaction with the CF-Network proteins -->

In [ ]:
cf_vertex.sort()
main_matrix = graph_main.get_adjacency()

In [ ]:
main_matrix_np = np.matrix(main_matrix)

In [ ]:
from scipy.stats import hypergeom
from scipy.stats import rankdata

def specific_neighbors(graph:ig.Graph,index:list):
    graph_matrix = np.matrix(graph.get_adjacency())
    neib = graph_matrix[:,index].sum(axis=1).transpose().tolist()[0] # the "CF-related" specific degree of each protein in the main graph
    gdegree = graph_matrix[:,:].sum(axis=1).transpose().tolist()[0] # the degree of each protein in the main graph
    multineibtab = pd.DataFrame(
        index=list(range(len(gdegree))),
        data={
            "CF Neighbors":neib,
            "Degree":gdegree,
        }   
    )
    c = len(index)
    N = graph_matrix.shape[0] - 1
    multineibtab["p(P value)"] = -1*np.log10(hypergeom.sf(multineibtab["CF Neighbors"],N,c,multineibtab["Degree"]))
    multineibtab["Rank"] = rankdata(multineibtab["p(P value)"])/graph.vcount()
    return multineibtab

main_cf_rank = specific_neighbors(graph_main,cf_vertex)

In [ ]:
def minimum_to_connect(main_graph, unconnected_graph_list, significant_node_df):
    nodes_to_add = significant_node_df.index.to_list()
    unconnected_nodes = list(unconnected_graph_list)
    connected_graph = main_graph.induced_subgraph(unconnected_nodes)
    number_of_components = len(connected_graph.components())

    print(f"There are {number_of_components} components in the unconnected graph")
    count = 0
    for node in nodes_to_add:
        if number_of_components == 1:
            break

        if node not in unconnected_nodes:
            temp_nodes = unconnected_nodes + [node]
            temp_graph = main_graph.induced_subgraph(temp_nodes)
            temp_components = len(temp_graph.components())

            if temp_components < number_of_components:
                count += 1
                unconnected_nodes.append(node)
                connected_graph = temp_graph
                number_of_components = temp_components
                print(f"Added node {node}, components: {number_of_components}")

    print(f"{count} nodes were added")
    return connected_graph

sig_interactors = main_cf_rank.loc[main_cf_rank["p(P value)"] >= -np.log10(0.05)].sort_values(axis=0,by="Rank")

test = minimum_to_connect(graph_main,unconnected_graph_list=cf_vertex,significant_node_df=sig_interactors)
test

main_cf_rank.loc[(main_cf_rank["p(P value)"] >= -np.log10(0.05))].sort_values(axis=0,by="Rank",ascending=False)


In [ ]:
# generate the csv files of each cluster for enrichment analysis
for cluster in range(len(cf_cluster)):
    nodes_in_cluster = cf_cluster[cluster]
    node_ids = pd.Series([nodes[node] for node in nodes_in_cluster])
    node_ids.to_csv(f"cf_cluster_{cluster}.csv")

cf_cluster_uniprot = []
for cluster in cf_cluster:
    cluster_id = pd.Series(cf_connected_graph.vs["name"])[cluster].to_list()
    cf_cluster_uniprot.append(cluster_id)
    
for cluster in cf_cluster_uniprot:
    if cftr in cluster:
        index = cf_cluster_uniprot.index(cluster)
        print(f"CFTR is in cluster {index} out of {len(cf_cluster_uniprot)}")

print([len(cluster) for cluster in cf_cluster_uniprot])


In [ ]:
cftr_cluster = cf_cluster_uniprot[index]

graph_indexes = {}
for protein in cftr_cluster:
    graph_index = cf_connected_graph.vs["name"].index(protein)
    graph_indexes[graph_index] = protein

cf_graph_indexes = pd.DataFrame(data={
    "index":graph_indexes.keys(),
    "UniprotID":graph_indexes.values()
},index=range(len(cftr_cluster)))

cf_graph_indexes

In [ ]:
from pyvis.network import Network

net = Network(notebook=True, directed=cf_connected_graph.is_directed())  # notebook=True for Jupyter

for vertex in cf_connected_graph.vs:
    net.add_node(vertex.index, label=str(vertex.index))  # Use vertex indices as node IDs

for edge in cf_connected_graph.es:
    net.add_edge(edge.source, edge.target)

In [ ]:
cf_expanded_graph = graph_main.induced_subgraph(union(main_cf_rank.sort_values(axis=0,by="Rank").head(250).index.tolist(),cf_vertex))

print("The percentage of connected proteins in this graph is",cf_expanded_graph.components().size(idx=which_max(cf_expanded_graph))*100/cf_expanded_graph.components().size(idx=which_max(cf_expanded_graph)),"%")#len(exclude(main_cf_rank.sort_values(axis=0,by="Rank").index.tolist(),cf_vertex))

print("The percentage of connected proteins in this graph is",cf_graph.components().size(idx=which_max(cf_graph))*100/len(cf_graph.vs),"%")

#print(graph_main.vs[exclude(unique(sig_interactors,cf_vertex),cf_vertex)]["name"])

In [ ]:
print(f'The expanded graph has {cf_expanded_graph.components().size(idx=which_max(cf_graph))} proteins')

In [ ]:
new_proteins = graph_main.vs[exclude(unique(sig_interactors),cf_vertex)]["name"]

new_prot_df = main_cf_rank.loc[exclude(unique(sig_interactors),cf_vertex),:]

new_prot_df['UniprotKD'] = new_proteins

new_prot_df.sort_values(axis=0,by="Rank")
#new_prot_df.to_csv("CF_significant_neighborhs.csv")

### Join the expanded DF with the core CF-related one to create a universe of CF-related proteins for enrichment analysis

In [ ]:
universe_name = graph_main.vs["name"]

universe_sr = pd.Series(universe_name,name ='UniprotKD')

universe_sr.to_csv("Whole_universe_main.csv")

In [ ]:
import requests
import os

def download_disgenet_file(url, save_path):
    """Downloads a file from DisGeNET."""
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded: {save_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")

# Example usage (replace with actual DisGeNET download URLs)
disgenet_urls = {
    "gene_disease_associations.tsv.gz": "YOUR_DISGENET_GENE_DISEASE_URL",
    "variant_disease_associations.tsv.gz": "YOUR_DISGENET_VARIANT_DISEASE_URL"
}

download_folder = "disgenet_data"
os.makedirs(download_folder, exist_ok=True)

for filename, url in disgenet_urls.items():
    save_path = os.path.join(download_folder, filename)
    download_disgenet_file(url, save_path)

In [ ]:
wb_data = pd.read_csv("wb_endo_quantification",sep="\t")

endo_log = [float(y) for y in wb_data.Log.tolist()[1:]]
print(endo_log)
time_span = [2.5,5,10]

def mean(lists):
    mean = sum(lists)/len(lists)
    return mean

def ln_reg(x, y, coef = False, Error = False):
    x_m = mean(x)
    y_m = mean(y)

    numerador = sum([(x-x_m)*(y-y_m) for x, y in zip(x,y)])
    denominador = sum([(x-x_m)**2 for x in x])

    coef_ang = numerador/denominador
    coef_lin = y_m - coef_ang*x_m
    regression = [coef_ang*x+coef_lin for x in x]
    
    S = (sum([(y-(coef_ang*x+coef_lin))**2 for x,y in zip(x,y)])/(len(y)-2))**0.5
    SE_coef_ang = S/(denominador**0.5)
    SE_coef_lin = S*(1/len(y) + x_m**2/denominador)**0.5

    if coef == True:
        coef_ang = coef_ang
        coef_lin = coef_lin
        return regression, coef_ang, coef_lin #Answer equals tuple (regression, coef_ang, coef_lin)
    elif Error == True:
        coef_ang = coef_ang
        coef_lin = coef_lin
        SE_coef_ang = SE_coef_ang
        SE_coef_lin = SE_coef_lin
        print(f'The coefficientes are a = {round(coef_ang,4)} ± {round(SE_coef_ang, 4)} and b = {round(coef_lin,4)} ± {round(SE_coef_lin, 4)}')
        return regression
    else:
        return regression
    
regression = ln_reg(time_span,endo_log,coef=True,Error=True)
print(regression)
fig = go.Figure()

fig.add_trace(go.Scatter(x=time_span,
                     y=endo_log,
                     mode="markers"
                     ))

fig.add_trace(go.Scatter(x=time_span,
                         y=regression[0],
                         mode="lines",
))

fig.update_layout(title='Log(Endocytosed CFTR)',
    width=800,  # Width in pixels
    height=600  # Height in pixels
)
fig.show()

# Colorectal Cancer (CRC)-related network

In [ ]:
crc = disgenet.loc[disgenet["diseaseName"] == "Colorectal Carcinoma"]

crc = crc["geneSymbol"].unique()

disgenet.loc[disgenet["diseaseName"] == "Colorectal Carcinoma"]

disgenet.loc[disgenet["diseaseName"] == "Colorectal Carcinoma"]["gene"].to_csv("crc_associated_genes.csv")

In [ ]:

import mygene

# Initialize the query object
mg = mygene.MyGeneInfo()

# Query mygene.info for UniProt ID mappings
gene_info = mg.querymany(crc, scopes='entrezgene', fields='uniprot', species='human')

# Extract GeneID to UniProt mapping
cf_uniprot = []
for entry in gene_info:
    gene_id = entry.get('query')
    uniprot_id = entry.get('uniprot', {}).get('Swiss-Prot')
    if uniprot_id not in cf_uniprot:
        cf_uniprot.append(uniprot_id)
    else:
        continue

In [ ]:
cf_uniprot